# BAGIAN 0: PENGATURAN PROYEK

## SEL 0.1: Hubungkan Google Drive & Buat Struktur Folder Proyek

In [ ]:
# =============================================================================
# SEL 0.1: Hubungkan Google Drive & Buat Struktur Folder Proyek
# =============================================================================
print("--- SEL 0.1: Memulai Inisiasi Proyek ---")

from google.colab import drive
import os

def setup_project_structure():
    """
    Menghubungkan Google Colab ke Google Drive dan membuat struktur folder
    yang diperlukan untuk proyek prediksi risiko kredit.

    Struktur Folder:
    - 00_data_raw: Untuk menyimpan dataset asli.
    - 01_data_processed: Untuk menyimpan data yang telah dibersihkan dan diproses.
    - 02_models: Untuk menyimpan file model yang telah dilatih.
    - 03_results: Untuk menyimpan hasil evaluasi, metrik, dan visualisasi.
    - 04_notebooks: (Opsional) Untuk menyimpan versi notebook ini.
    """
    try:
        # 1. Hubungkan ke Google Drive
        print("📂 Menghubungkan ke Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("   -> Google Drive berhasil terhubung.")

        # 2. Definisikan Path Proyek Utama
        # Ganti 'Credit_Risk_Prediction' jika Anda ingin nama folder yang berbeda
        PROJECT_ROOT = "/content/drive/MyDrive/Colab_Projects/Credit_Risk_Prediction#1"
        print(f"\n📁 Direktori utama proyek diatur ke: {PROJECT_ROOT}")

        # 3. Definisikan dan Buat Struktur Sub-folder
        subdirectories = {
            "DATA_RAW_DIR": "00_data_raw",
            "DATA_PROCESSED_DIR": "01_data_processed",
            "MODELS_DIR": "02_models",
            "RESULTS_DIR": "03_results",
            "NOTEBOOKS_DIR": "04_notebooks"
        }

        print("\n🛠️  Membuat struktur folder di Google Drive...")
        os.makedirs(PROJECT_ROOT, exist_ok=True)

        # Membuat setiap sub-folder dan menyimpannya sebagai variabel global
        for dir_variable, dir_path in subdirectories.items():
            full_path = os.path.join(PROJECT_ROOT, dir_path)
            os.makedirs(full_path, exist_ok=True)
            globals()[dir_variable] = full_path
            print(f"   -> Folder '{dir_path}' siap di: {full_path}")

        print("\n✅ Inisiasi Proyek Selesai. Semua struktur folder telah siap.")
        return True

    except Exception as e:
        print(f"\n❌ GAGAL saat inisiasi proyek. Error: {e}")
        return False

# Jalankan fungsi setup
setup_project_structure()


## SEL 0.2: Instalasi dan Impor Pustaka

In [ ]:
# =============================================================================
# SEL 0.2: Instalasi dan Impor Pustaka
# =============================================================================
print("--- SEL 0.2: Memulai Instalasi dan Impor Pustaka ---")

import sys

# --- 1. Instalasi Pustaka yang Dibutuhkan ---
# Kita menggunakan --quiet untuk menjaga output tetap bersih.
# CatBoost, XGBoost, dan LightGBM adalah model boosting yang akan kita gunakan.
# SHAP diperlukan untuk interpretasi model (XAI).
print("📦 Menginstal pustaka yang diperlukan (catboost, xgboost, lightgbm, shap)...")
!{sys.executable} -m pip install pandas numpy scikit-learn matplotlib seaborn catboost xgboost lightgbm shap --quiet
print("   -> Instalasi selesai.")

# --- 2. Impor Pustaka Utama ---
print("\n📚 Mengimpor pustaka-pustaka utama...")

# Pustaka Dasar untuk Manipulasi Data
import pandas as pd
import numpy as np

# Pustaka untuk Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Pustaka Scikit-learn untuk Preprocessing, Pemodelan, dan Evaluasi
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    roc_curve,
    classification_report
)

# Pustaka untuk Model Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Pustaka untuk Interpretasi Model
import shap

# Pustaka Utilitas
import os
import joblib # Untuk menyimpan dan memuat model/pipeline
import warnings

# --- 3. Konfigurasi Lingkungan ---
print("\n⚙️  Melakukan konfigurasi lingkungan...")

# Mengatur agar semua kolom Pandas ditampilkan
pd.set_option('display.max_columns', None)

# Mengatur gaya plot default
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-whitegrid')


# Menonaktifkan peringatan yang tidak krusial untuk menjaga kebersihan output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

print("\n✅ Semua pustaka telah diimpor dan lingkungan telah dikonfigurasi.")


# BAGIAN 1: PEMUATAN DATA & PEMERIKSAAN AWAL

## SEL 1.1: Muat Dataset dari Google Drive

In [ ]:
# =============================================================================
# SEL 1.1: Muat Dataset dari Google Drive
# =============================================================================
print("--- SEL 1.1: Memulai Pemuatan Dataset ---")

# Pastikan variabel path dari SEL 0.1 tersedia
if 'DATA_RAW_DIR' not in globals():
    print("❌ ERROR: Variabel path 'DATA_RAW_DIR' tidak ditemukan.")
    print("   -> Harap jalankan SEL 0.1 untuk menginisialisasi struktur proyek terlebih dahulu.")
else:
    try:
        # --- 1. Definisikan Lokasi dan Nama File ---
        # Ganti 'loan_data_2007_2014.csv' jika nama file Anda berbeda
        file_name = "loan_data_2007_2014.csv"
        file_path = os.path.join(DATA_RAW_DIR, file_name)
        print(f"📖 Mencoba memuat file dari: {file_path}")

        # --- 2. Periksa Keberadaan File ---
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File '{file_name}' tidak ditemukan di direktori '00_data_raw'. "
                                    "Harap pastikan file sudah diunggah ke Google Drive.")

        # --- 3. Muat Data ke DataFrame ---
        df_raw = pd.read_csv(file_path)
        print("\n✅ Dataset berhasil dimuat ke dalam DataFrame 'df_raw'.")

    except Exception as e:
        print(f"\n❌ GAGAL memuat dataset. Error: {e}")



## SEL 1.2: Pemeriksaan Awal Struktur Data

In [ ]:
# =============================================================================
# SEL 1.2: Pemeriksaan Awal Struktur Data (Disempurnakan)
# =============================================================================
print("--- SEL 1.2: Melakukan Pemeriksaan Awal Struktur Data ---")

# Pastikan DataFrame 'df_raw' sudah ada
if 'df_raw' in globals():
    try:
        # --- 1. Tampilkan Dimensi DataFrame ---
        print(f"Bentuk (Dimensi) Data: {df_raw.shape[0]} baris dan {df_raw.shape[1]} kolom.")

        # --- 2. Tampilkan 5 Baris Pertama ---
        print("\n--- 5 Baris Pertama Data ---")
        display(df_raw.head())

        # --- 3. Tampilkan 5 Baris Terakhir ---
        print("\n--- 5 Baris Terakhir Data ---")
        display(df_raw.tail())

        # --- 4. Fungsi Ringkasan Komprehensif (Saran Tambahan) ---
        def summarize_df(df):
            """
            Membuat ringkasan DataFrame yang mencakup tipe data, nilai hilang,
            persentase hilang, jumlah nilai unik, dan contoh nilai.
            """
            print("\n--- Ringkasan Komprehensif DataFrame ---")
            summary = pd.DataFrame({
                'Dtype': df.dtypes,
                'Missing Values': df.isnull().sum(),
                'Missing %': (df.isnull().sum() / len(df)) * 100,
                'N-Unique': df.nunique(),
                'Sample Values': [df[col].dropna().unique()[:3] for col in df.columns]
            })
            return summary

        # Tampilkan ringkasan menggunakan fungsi yang baru dibuat
        df_summary = summarize_df(df_raw)
        display(df_summary)

        # --- 5. Tampilkan Informasi Umum (Tipe Data & Nilai Non-Null) ---
        # Ini tetap berguna untuk melihat penggunaan memori
        print("\n--- Informasi Umum DataFrame (Tipe Data & Penggunaan Memori) ---")
        df_raw.info(verbose=False) # Menggunakan verbose=False karena detail sudah ada di ringkasan

    except Exception as e:
        print(f"\n❌ GAGAL melakukan pemeriksaan awal. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_raw' tidak ditemukan. Harap jalankan SEL 1.1 terlebih dahulu.")



# BAGIAN 2: ANALISIS DATA EKSPLORATIF (EDA)

## SEL 2.1: Analisis Statistik Deskriptif & Identifikasi Nilai Hilang

In [ ]:
# =============================================================================
# SEL 2.1: Analisis Statistik Deskriptif & Identifikasi Nilai Hilang
# =============================================================================
print("--- SEL 2.1: Memulai Analisis Statistik Deskriptif & Nilai Hilang ---")

# Pastikan DataFrame 'df_raw' sudah ada
if 'df_raw' in globals():
    try:
        # --- 1. Statistik Deskriptif untuk Kolom Numerik ---
        print("\n--- Statistik Deskriptif (Kolom Numerik) ---")
        # Menggunakan .T (transpose) agar lebih mudah dibaca
        display(df_raw.describe().T)

        # --- 2. Menghitung Persentase Nilai Hilang (Missing Values) ---
        print("\n--- Persentase Nilai Hilang per Kolom ---")
        missing_values = df_raw.isnull().sum()
        missing_percentage = (missing_values / len(df_raw)) * 100

        # Gabungkan jumlah dan persentase untuk tampilan yang lebih baik
        missing_info = pd.DataFrame({
            'Jumlah Hilang': missing_values,
            'Persentase Hilang (%)': missing_percentage
        })

        # Tampilkan hanya kolom yang memiliki nilai hilang, diurutkan dari yang terbesar
        missing_info = missing_info[missing_info['Jumlah Hilang'] > 0].sort_values(by='Persentase Hilang (%)', ascending=False)

        if missing_info.empty:
            print("✅ Tidak ada nilai yang hilang dalam dataset.")
        else:
            print(f"Ditemukan {len(missing_info)} kolom dengan nilai yang hilang.")
            display(missing_info)

    except Exception as e:
        print(f"\n❌ GAGAL melakukan analisis. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_raw' tidak ditemukan. Harap jalankan SEL 1.1 & 1.2 terlebih dahulu.")


## SEL 2.2: Analisis Variabel Target (loan_status)

In [ ]:
# =============================================================================
# SEL 2.2: Analisis Variabel Target (loan_status)
# =============================================================================
print("--- SEL 2.2: Memulai Analisis Variabel Target ---")

if 'df_raw' in globals():
    print("\n--- Distribusi Nilai pada Kolom 'loan_status' ---")
    status_counts = df_raw['loan_status'].value_counts(normalize=True) * 100

    plt.figure(figsize=(10, 6))
    sns.barplot(x=status_counts.index, y=status_counts.values, palette='mako')
    plt.title('Distribusi Status Pinjaman (Loan Status)')
    plt.ylabel('Persentase (%)')
    plt.xlabel('Status Pinjaman')
    plt.xticks(rotation=45, ha='right')
    plt.show()

    display(status_counts.to_frame(name='Persentase (%)'))
else:
    print("❌ ERROR: DataFrame 'df_raw' tidak ditemukan.")


## SEL 2.3 (BARU): Analisis Univariat Komprehensif

In [ ]:
# =============================================================================
# SEL 2.3 (BARU): Analisis Univariat Komprehensif (dengan Penyimpanan)
# =============================================================================
print("--- SEL 2.3: Memulai Analisis Univariat (Distribusi Setiap Fitur) ---")

if 'df_raw' in globals() and 'RESULTS_DIR' in globals():
    # --- Persiapan Folder Penyimpanan ---
    VIZ_DIR = os.path.join(RESULTS_DIR, "visualizations/univariat")
    os.makedirs(VIZ_DIR, exist_ok=True)
    print(f"🖼️  Visualisasi akan disimpan di: {VIZ_DIR}")

    # Pisahkan kolom numerik dan kategorikal untuk analisis
    numeric_cols = df_raw.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df_raw.select_dtypes(include=['object', 'category']).columns.tolist()

    cols_to_exclude = ['id', 'member_id', 'loan_status']
    numeric_cols = [col for col in numeric_cols if col not in cols_to_exclude]
    categorical_cols = [col for col in categorical_cols if col not in cols_to_exclude]

    print(f"Ditemukan {len(numeric_cols)} fitur numerik dan {len(categorical_cols)} fitur kategorikal untuk dianalisis.")

    # --- 1. Fungsi untuk Analisis Univariat Numerik (dengan Penyimpanan) ---
    def plot_numeric_distribution(df, col_list):
        print(f"\n--- Menganalisis Distribusi {len(col_list)} Fitur Numerik ---")
        for col in col_list:
            fig = plt.figure(figsize=(14, 4))

            plt.subplot(1, 2, 1)
            sns.histplot(df[col], kde=True, bins=50)
            plt.title(f'Histogram Distribusi - {col}')

            plt.subplot(1, 2, 2)
            sns.boxplot(x=df[col])
            plt.title(f'Boxplot - {col}')

            plt.tight_layout()

            # Simpan plot ke file
            file_path = os.path.join(VIZ_DIR, f"univariate_dist_{col}.png")
            plt.savefig(file_path, dpi=150, bbox_inches='tight')
            plt.show()
            print(f"   -> Plot untuk '{col}' disimpan ke {file_path}")

    # --- 2. Fungsi untuk Analisis Univariat Kategorikal (dengan Penyimpanan) ---
    def plot_categorical_distribution(df, col_list, top_n=20):
        print(f"\n--- Menganalisis Distribusi {len(col_list)} Fitur Kategorikal ---")
        for col in col_list:
            fig = plt.figure(figsize=(10, 5))

            title_suffix = ""
            if df[col].nunique() > top_n:
                top_categories = df[col].value_counts().nlargest(top_n).index
                sns.countplot(y=col, data=df[df[col].isin(top_categories)], order=top_categories, palette='mako')
                title_suffix = f" (Top {top_n})"
            else:
                sns.countplot(y=col, data=df, order=df[col].value_counts().index, palette='mako')

            plt.title(f'Distribusi Fitur - {col}{title_suffix}')
            plt.tight_layout()

            # Simpan plot ke file
            file_path = os.path.join(VIZ_DIR, f"univariate_count_{col}.png")
            plt.savefig(file_path, dpi=150, bbox_inches='tight')
            plt.show()
            print(f"   -> Plot untuk '{col}' disimpan ke {file_path}")

    # Jalankan fungsi analisis pada beberapa kolom kunci
    key_numeric_to_plot = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_util', 'total_acc']
    key_categorical_to_plot = ['grade', 'sub_grade', 'home_ownership', 'verification_status', 'purpose', 'term']

    plot_numeric_distribution(df_raw, [col for col in key_numeric_to_plot if col in df_raw.columns])
    plot_categorical_distribution(df_raw, [col for col in key_categorical_to_plot if col in df_raw.columns])

else:
    print("❌ ERROR: DataFrame 'df_raw' atau path 'RESULTS_DIR' tidak ditemukan.")


## SEL 2.4: Analisis Bivariat (Fitur vs. Target) (dengan Penyimpanan)

In [ ]:
# =============================================================================
# SEL 2.4: Analisis Bivariat (Fitur vs. Target) (dengan Penyimpanan)
# =============================================================================
print("--- SEL 2.4: Memulai Analisis Bivariat (Hubungan Fitur dengan Risiko Kredit) ---")

if 'df_raw' in globals() and 'RESULTS_DIR' in globals():
    # Pastikan folder penyimpanan ada
    VIZ_DIR = os.path.join(RESULTS_DIR, "visualizations/bivariat")
    os.makedirs(VIZ_DIR, exist_ok=True)

    df_eda = df_raw.copy()
    bad_loan_statuses = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'Late (31-120 days)']
    df_eda['credit_risk_temp'] = np.where(df_eda['loan_status'].isin(bad_loan_statuses), 1, 0)

    print(f"Proporsi 'Bad Loan' (Risiko=1) dalam data: {df_eda['credit_risk_temp'].mean() * 100:.2f}%")

    # --- 1. Analisis Fitur Kategorikal vs. Target (dengan Penyimpanan) ---
    print("\n--- Menganalisis Hubungan Fitur Kategorikal dengan Risiko Kredit ---")
    for col in ['grade', 'term', 'home_ownership', 'verification_status']:
        if col in df_eda.columns:
            fig = plt.figure(figsize=(10, 5))
            risk_by_cat = df_eda.groupby(col)['credit_risk_temp'].mean().sort_values()
            sns.barplot(x=risk_by_cat.index, y=risk_by_cat.values, palette='viridis')
            plt.title(f'Tingkat Risiko Rata-rata per {col}')
            plt.ylabel('Rata-rata Risiko (Bad Loan Rate)')
            plt.xticks(rotation=45, ha='right')

            file_path = os.path.join(VIZ_DIR, f"bivariate_risk_vs_{col}.png")
            plt.savefig(file_path, dpi=150, bbox_inches='tight')
            plt.show()
            print(f"   -> Plot untuk '{col}' disimpan ke {file_path}")

    # --- 2. Analisis Fitur Numerik vs. Target (dengan Penyimpanan) ---
    print("\n--- Menganalisis Hubungan Fitur Numerik dengan Risiko Kredit ---")
    for col in ['int_rate', 'annual_inc', 'dti', 'loan_amnt']:
        if col in df_eda.columns:
            fig = plt.figure(figsize=(8, 5))
            plot_col_name = col
            if col == 'annual_inc':
                sns.boxplot(x='credit_risk_temp', y=np.log1p(df_eda[col]), data=df_eda, palette='pastel')
                plt.ylabel(f'log({col})')
                plot_col_name = f"log_{col}"
            else:
                sns.boxplot(x='credit_risk_temp', y=col, data=df_eda, palette='pastel')
                plt.ylabel(col)

            plt.title(f'Distribusi {col} berdasarkan Risiko Kredit')
            plt.xticks([0, 1], ['Good Loan', 'Bad Loan'])

            file_path = os.path.join(VIZ_DIR, f"bivariate_box_{plot_col_name}_vs_risk.png")
            plt.savefig(file_path, dpi=150, bbox_inches='tight')
            plt.show()
            print(f"   -> Plot untuk '{col}' disimpan ke {file_path}")
else:
    print("❌ ERROR: DataFrame 'df_raw' atau path 'RESULTS_DIR' tidak ditemukan.")


## SEL 2.5: Analisis Korelasi Mendalam (dengan Penyimpanan)

In [ ]:
# =============================================================================
# SEL 2.5: Analisis Korelasi Mendalam (dengan Penyimpanan)
# =============================================================================
print("--- SEL 2.5: Memulai Analisis Korelasi Mendalam ---")

if 'df_eda' in globals() and 'RESULTS_DIR' in globals():
    # --- Persiapan Folder Penyimpanan ---
    VIZ_DIR = os.path.join(RESULTS_DIR, "visualizations/korelasi")
    DATA_DIR = os.path.join(RESULTS_DIR, "analysis_data")
    os.makedirs(VIZ_DIR, exist_ok=True)
    os.makedirs(DATA_DIR, exist_ok=True)
    print(f"🖼️  Visualisasi akan disimpan di: {VIZ_DIR}")
    print(f"📊 Data analisis akan disimpan di: {DATA_DIR}")

    numeric_df = df_eda.select_dtypes(include=np.number)
    correlation_matrix = numeric_df.corr()

    # --- 1. Heatmap Korelasi (dengan Penyimpanan) ---
    fig = plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False)
    plt.title('Heatmap Korelasi Fitur Numerik')

    file_path_viz = os.path.join(VIZ_DIR, "correlation_heatmap.png")
    plt.savefig(file_path_viz, dpi=150, bbox_inches='tight')
    plt.show()
    print(f"   -> Heatmap korelasi disimpan ke {file_path_viz}")

    # --- 2. Korelasi Tertinggi dengan Target (dengan Penyimpanan) ---
    print("\n--- Fitur dengan Korelasi Tertinggi terhadap 'credit_risk_temp' ---")
    corr_with_target = correlation_matrix['credit_risk_temp'].sort_values(ascending=False).drop('credit_risk_temp')

    # Simpan hasil ke CSV
    file_path_data = os.path.join(DATA_DIR, "correlation_with_target.csv")
    corr_with_target.to_csv(file_path_data, header=['correlation'])
    print(f"   -> Data korelasi dengan target disimpan ke {file_path_data}")

    display(corr_with_target.to_frame().head(10))
    display(corr_with_target.to_frame().tail(10))

    # --- 3. Identifikasi Multikolinearitas Tinggi (dengan Penyimpanan) ---
    print("\n--- Pasangan Fitur dengan Multikolinearitas Tinggi (Korelasi > 0.7) ---")
    abs_corr_matrix = correlation_matrix.abs()
    corr_pairs = abs_corr_matrix.unstack()
    sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)
    high_corr_pairs = sorted_pairs[(sorted_pairs > 0.7) & (sorted_pairs < 1)]

    if high_corr_pairs.empty:
        print("✅ Tidak ditemukan pasangan fitur dengan korelasi absolut > 0.7.")
    else:
        print("Ditemukan pasangan fitur yang sangat berkorelasi (potensi redundan):")
        high_corr_df = high_corr_pairs.to_frame(name='Correlation').reset_index()
        high_corr_df.columns = ['Feature 1', 'Feature 2', 'Correlation']

        # Simpan hasil ke CSV
        file_path_data = os.path.join(DATA_DIR, "high_multicollinearity_pairs.csv")
        high_corr_df.to_csv(file_path_data, index=False)
        print(f"   -> Data pasangan multikolinearitas tinggi disimpan ke {file_path_data}")

        display(high_corr_df)

else:
    print("❌ ERROR: DataFrame 'df_eda' atau path 'RESULTS_DIR' tidak ditemukan.")


## SEL 2.6: Ringkasan EDA & Rencana Aksi (Dinamis dengan Python)

In [ ]:
# =============================================================================
# SEL 2.6: Ringkasan EDA & Rencana Aksi (Dinamis dengan Python) - (FIXED)
# =============================================================================
print("--- SEL 2.6: Membuat Ringkasan EDA dan Rencana Aksi ---")

class EdaSummary:
    def __init__(self):
        self.findings = {
            "Target Variable": [],
            "Columns to Drop": [],
            "Feature Engineering Ideas": [],
            "Preprocessing Strategy": []
        }
        self.log = []

    def add_finding(self, category, description):
        if category in self.findings:
            self.findings[category].append(description)
        else:
            print(f"Warning: Kategori '{category}' tidak ditemukan.")

    def generate_report(self):
        print("\n" + "="*60)
        print("      RINGKASAN EKSEKUTIF & RENCANA AKSI (Berdasarkan EDA)")
        print("="*60 + "\n")

        for category, items in self.findings.items():
            print(f"### {category.upper()} ###")
            if not items:
                print("   (Belum ada temuan)")
            for i, item in enumerate(items, 1):
                print(f"   {i}. {item}")
            print("-" * 60)

        print("\n### LOG TEMUAN OTOMATIS ###")
        if not self.log:
            print("   (Tidak ada log otomatis yang dihasilkan)")
        for entry in self.log:
            print(f"   - {entry}")
        print("="*60)

# --- Proses Pengisian Ringkasan Secara Dinamis ---
# Pastikan variabel dari sel sebelumnya ada
if 'df_raw' in globals() and 'missing_info' in globals() and 'high_corr_pairs' in globals():
    summary = EdaSummary()

    # 1. Analisis Variabel Target
    summary.add_finding("Target Variable", "Definisi 'Bad Loan' (risk=1) adalah status: 'Charged Off', 'Default', 'Late (31-120 days)', dll.")
    summary.add_finding("Target Variable", "Definisi 'Good Loan' (risk=0) adalah 'Fully Paid'.")
    summary.add_finding("Target Variable", "Status pinjaman yang masih berjalan ('Current', 'In Grace Period') akan difilter dan tidak digunakan dalam pemodelan.")

    # 2. Analisis Kolom yang Akan Dihapus (Drop)
    # Berdasarkan nilai hilang
    high_missing_cols = missing_info[missing_info['Persentase Hilang (%)'] > 40].index.tolist()
    if high_missing_cols:
        summary.add_finding("Columns to Drop", f"Kolom dengan >40% nilai hilang: {', '.join(high_missing_cols)}.")
        summary.log.append(f"LOG: Terdeteksi {len(high_missing_cols)} kolom dengan nilai hilang > 40%.")

    # Berdasarkan relevansi/kebocoran data (manual)
    leakage_cols = ['funded_amnt', 'funded_amnt_inv', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'last_pymnt_d', 'last_pymnt_amnt']
    summary.add_finding("Columns to Drop", f"Kolom yang mengandung kebocoran data (informasi masa depan): {', '.join(leakage_cols)}.")

    # Berdasarkan multikolinearitas
    if not high_corr_pairs.empty:
        # Logika sederhana: simpan yang pertama, tandai yang kedua untuk dihapus
        cols_to_drop_corr = list(set([idx[1] for idx in high_corr_pairs.index]))
        summary.add_finding("Columns to Drop", f"Kolom redundan karena multikolinearitas tinggi (>0.7): {', '.join(cols_to_drop_corr)} (contoh: 'loan_amnt' vs 'funded_amnt').")
        summary.log.append(f"LOG: Terdeteksi {len(cols_to_drop_corr)} kolom redundan karena korelasi tinggi.")

    # Kolom dengan varians rendah atau tidak relevan
    summary.add_finding("Columns to Drop", "Kolom ID ('id', 'member_id') dan teks bebas ('desc', 'title') akan dihapus.")

    # 3. Rencana Feature Engineering
    summary.add_finding("Feature Engineering Ideas", "Buat 'credit_history_length' dari 'issue_d' dan 'earliest_cr_line'.")
    # INI BARIS YANG DIPERBAIKI: addfinding -> add_finding
    summary.add_finding("Feature Engineering Ideas", "Buat rasio-rasio penting: 'loan_to_income', 'installment_to_income'.")
    summary.add_finding("Feature Engineering Ideas", "Terapkan transformasi log pada fitur numerik yang miring (skewed) seperti 'annual_inc' dan 'revol_bal'.")

    # 4. Rencana Preprocessing
    summary.add_finding("Preprocessing Strategy", "Imputasi nilai hilang pada fitur numerik menggunakan median (lebih robust terhadap outlier).")
    summary.add_finding("Preprocessing Strategy", "Imputasi nilai hilang pada fitur kategorikal menggunakan modus ('missing' atau nilai paling umum).")
    summary.add_finding("Preprocessing Strategy", "Terapkan StandardScaler pada semua fitur numerik setelah imputasi.")
    summary.add_finding("Preprocessing Strategy", "Terapkan OneHotEncoder pada fitur kategorikal setelah imputasi.")
    summary.add_finding("Preprocessing Strategy", "Gunakan ColumnTransformer untuk memastikan semua langkah preprocessing digabungkan dalam satu pipeline yang konsisten.")

    # Tampilkan laporan yang dihasilkan
    summary.generate_report()

else:
    print("❌ ERROR: Pastikan semua sel EDA sebelumnya (2.1 - 2.5) telah dijalankan untuk menghasilkan variabel yang dibutuhkan (df_raw, missing_info, high_corr_pairs).")



# BAGIAN 3A: PRE-PROCESSING & FEATURE ENGINEERING AWAL (SEBELUM SPLIT)

## SEL 3A.1 (REVISI): Basic Feature Engineering & Row Filtering

In [ ]:
# =============================================================================
# SEL 3A.1 (REVISI): Basic Feature Engineering & Row Filtering
# =============================================================================
print("--- SEL 3A.1: Memulai Basic Feature Engineering & Row Filtering ---")

if 'df_raw' in globals():
    try:
        # --- 1. Buat DataFrame Kerja Utama ---
        df_cleaned = df_raw.copy()
        print("✅ Salinan DataFrame 'df_raw' telah dibuat sebagai 'df_cleaned'.")

        # --- 2. Basic Feature Engineering ---
        print("\n🛠️  Memulai pembuatan fitur dasar...")

        # a. Fitur Berbasis Tanggal (dengan logika yang sudah benar)
        df_cleaned['issue_d_dt'] = pd.to_datetime(df_cleaned['issue_d'], format='%b-%y', errors='coerce')

        def smart_date_converter(row):
            issue_date = row['issue_d_dt']
            cr_line_str = row['earliest_cr_line']
            if pd.isna(issue_date) or pd.isna(cr_line_str): return pd.NaT
            cr_line_date = pd.to_datetime(cr_line_str, format='%b-%y', errors='coerce')
            if cr_line_date > issue_date: return cr_line_date - pd.DateOffset(years=100)
            return cr_line_date

        df_cleaned['earliest_cr_line_dt'] = df_cleaned.apply(smart_date_converter, axis=1)
        df_cleaned['credit_history_length_years'] = ((df_cleaned['issue_d_dt'] - df_cleaned['earliest_cr_line_dt']).dt.days / 365.25)
        print("   -> Fitur 'credit_history_length_years' dibuat.")

        # b. Ekstraksi Numerik dari 'term'
        df_cleaned['term_in_months'] = df_cleaned['term'].str.extract(r'(\d+)').astype(float)
        print("   -> Fitur 'term_in_months' dibuat.")

        # --- 3. Pembuatan Target & Pemfilteran Baris ---
        print("\n🎯 Membuat variabel target dan memfilter baris...")
        bad_loan_statuses = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'Late (31-120 days)']
        df_cleaned['credit_risk'] = np.where(df_cleaned['loan_status'].isin(bad_loan_statuses), 1, 0)

        final_statuses = ['Fully Paid'] + bad_loan_statuses
        original_rows = len(df_cleaned)
        df_cleaned = df_cleaned[df_cleaned['loan_status'].isin(final_statuses)]
        print(f"   -> Baris difilter untuk status final. Jumlah baris: {len(df_cleaned)} (dari {original_rows}).")

        print("\n✅ SEL 3A.1 Selesai. 'df_cleaned' siap untuk langkah selanjutnya.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.1. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_raw' tidak ditemukan.")


## SEL 3A.2 (REVISI): Advanced Feature Engineering

In [ ]:
# =============================================================================
# SEL 3A.2 (REVISI FINAL & TERLENGKAP): Comprehensive Feature Engineering
# =============================================================================
print("--- SEL 3A.2: Memulai Comprehensive Feature Engineering ---")

if 'df_cleaned' in globals():
    try:
        # --- 1. Ordinal Encoding untuk Grade (diperlukan untuk fitur interaksi) ---
        print("   -> Menerapkan Ordinal Encoding untuk 'grade' dan 'sub_grade'...")
        grade_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
        df_cleaned['grade_numeric'] = df_cleaned['grade'].map(grade_map)

        sub_grade_sorted = sorted(df_cleaned['sub_grade'].dropna().unique())
        sub_grade_map = {sub_grade: i for i, sub_grade in enumerate(sub_grade_sorted)}
        df_cleaned['sub_grade_numeric'] = df_cleaned['sub_grade'].map(sub_grade_map)

        # --- 2. Membuat Semua Fitur yang Direncanakan ---
        print("   -> Membuat fitur rasio utang dan pendapatan...")
        df_cleaned['loan_to_income_ratio'] = (df_cleaned['loan_amnt'] / df_cleaned['annual_inc']).replace([np.inf, -np.inf], np.nan)
        df_cleaned['installment_to_income_ratio'] = (df_cleaned['installment'] / (df_cleaned['annual_inc'] / 12)).replace([np.inf, -np.inf], np.nan)
        df_cleaned['total_balance_to_income_ratio'] = (df_cleaned['tot_cur_bal'] / df_cleaned['annual_inc']).replace([np.inf, -np.inf], np.nan)

        print("   -> Membuat fitur perilaku kredit...")

        # --- LOGIKA CERDAS UNTUK DELINQUENCY RATE ---
        # Hitung rasio seperti biasa
        rate = df_cleaned['delinq_2yrs'] / df_cleaned['total_acc']
        # Ganti inf (hasil dari pembagian dengan 0) dengan 0, karena jika total_acc=0, delinq_rate juga 0.
        # Biarkan NaN tetap NaN untuk diimputasi nanti.
        df_cleaned['delinquency_rate'] = rate.replace([np.inf, -np.inf], 0)
        # -------------------------------------------

        df_cleaned['revolving_balance_to_limit_ratio'] = (df_cleaned['revol_bal'] / df_cleaned['total_rev_hi_lim']).replace([np.inf, -np.inf], np.nan)
        df_cleaned['accounts_per_year_of_credit'] = (df_cleaned['total_acc'] / df_cleaned['credit_history_length_years']).replace([np.inf, -np.inf], np.nan)

        print("   -> Membuat fitur interaksi...")
        df_cleaned['term_x_int_rate'] = df_cleaned['term_in_months'] * df_cleaned['int_rate']
        df_cleaned['grade_x_annual_inc'] = df_cleaned['grade_numeric'] * df_cleaned['annual_inc']

        print("\n✅ Comprehensive Feature Engineering selesai.")
        print(f"   -> Dimensi DataFrame sekarang: {df_cleaned.shape}")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.2. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Jalankan SEL 3A.1 terlebih dahulu.")


## SEL 3A.3 (REVISI): Final Column Cleaning

In [ ]:
# =============================================================================
# SEL 3A.3 (REVISI): Final Column Cleaning
# =============================================================================
print("--- SEL 3A.3: Memulai Pembersihan Kolom Final ---")

if 'df_cleaned' in globals():
    try:
        # --- 1. Definisikan Semua Kolom yang Akan Dihapus ---
        # Kolom dengan >40% nilai hilang (berdasarkan EDA awal)
        cols_to_drop_missing = ['inq_fi', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_last_12m', 'annual_inc_joint', 'verification_status_joint', 'dti_joint', 'total_cu_tl', 'il_util', 'mths_since_rcnt_il', 'total_bal_il', 'open_il_24m', 'open_il_12m', 'open_il_6m', 'open_acc_6m', 'open_rv_12m', 'mths_since_last_record', 'mths_since_last_major_derog', 'desc', 'mths_since_last_delinq', 'next_pymnt_d']

        # Kolom yang mengandung kebocoran data
        cols_to_drop_leakage = ['funded_amnt', 'funded_amnt_inv', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'last_pymnt_d', 'last_pymnt_amnt', 'recoveries', 'collection_recovery_fee', 'out_prncp', 'out_prncp_inv', 'total_pymnt_inv']

        # Kolom ID, teks bebas, dan varians rendah
        cols_to_drop_other = ['id', 'member_id', 'url', 'title', 'zip_code', 'emp_title', 'addr_state', 'policy_code', 'pymnt_plan', 'Unnamed: 0']

        # Kolom yang sudah digunakan untuk membuat fitur baru
        cols_to_drop_used_for_fe = ['issue_d', 'earliest_cr_line', 'issue_d_dt', 'earliest_cr_line_dt', 'loan_status', 'term', 'grade', 'sub_grade', 'emp_length']

        # Gabungkan semua daftar dan hapus duplikat
        total_cols_to_drop = list(set(cols_to_drop_missing + cols_to_drop_leakage + cols_to_drop_other + cols_to_drop_used_for_fe))

        # Filter hanya kolom yang benar-benar ada di DataFrame
        total_cols_to_drop = [col for col in total_cols_to_drop if col in df_cleaned.columns]

        # --- 2. Hapus Kolom ---
        df_cleaned.drop(columns=total_cols_to_drop, inplace=True)

        print(f"✅ {len(total_cols_to_drop)} kolom yang tidak diperlukan telah dihapus.")
        print(f"   -> Dimensi DataFrame sekarang: {df_cleaned.shape}")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.3. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Jalankan SEL 3A.2 terlebih dahulu.")


In [ ]:
# =============================================================================
# SEL 3A.3.1: Investigasi Dampak Penghapusan Outlier
# =============================================================================
print("--- SEL 3A.4: Memulai Investigasi Dampak Penghapusan Outlier ---")

# Kita akan melakukan ini pada df_cleaned sebelum di-split
# untuk mendapatkan gambaran umum.
if 'df_cleaned' in globals():
    try:
        # --- 1. Pilih Fitur Numerik Kunci untuk Diperiksa ---
        features_to_check = [
            'annual_inc', 'dti', 'revol_bal', 'open_acc', 'total_acc',
            'credit_history_length_years', 'loan_amnt', 'int_rate'
        ]
        # Pastikan semua fitur ada di dataframe
        features_to_check = [f for f in features_to_check if f in df_cleaned.columns]
        print(f"Fitur yang akan diperiksa untuk outlier: {features_to_check}")

        # --- 2. Hitung Batas IQR untuk Setiap Fitur ---
        outlier_indices = set()

        for col in features_to_check:
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR

            # Temukan indeks dari baris yang mengandung outlier untuk kolom ini
            col_outliers = df_cleaned[
                (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)
            ].index

            print(f"   -> Kolom '{col}': Ditemukan {len(col_outliers)} outlier (Batas: [{lower_bound:.2f}, {upper_bound:.2f}])")

            # Tambahkan indeks outlier ke set utama
            outlier_indices.update(col_outliers)

        # --- 3. Hitung Dampak Total ---
        total_outlier_rows = len(outlier_indices)
        total_rows = len(df_cleaned)
        percentage_impact = (total_outlier_rows / total_rows) * 100

        print("\n--- Ringkasan Dampak Penghapusan Outlier ---")
        print(f"Total baris dalam dataset: {total_rows}")
        print(f"Jumlah baris unik yang mengandung setidaknya satu outlier: {total_outlier_rows}")
        print(f"Persentase data yang akan dihapus: {percentage_impact:.2f}%")

        # --- 4. Berikan Rekomendasi Berbasis Data ---
        print("\n--- Rekomendasi ---")
        if percentage_impact < 5:
            print("Dampak penghapusan < 5%. Menghapus baris adalah opsi yang sangat layak dan bersih.")
            print("Ini akan menjaga keaslian distribusi data tanpa kehilangan terlalu banyak informasi.")
        elif percentage_impact < 15:
            print("Dampak penghapusan cukup signifikan (5-15%). Menghapus baris akan menyebabkan kehilangan informasi yang cukup besar.")
            print("Disarankan untuk mempertimbangkan Winsorizing atau metode penanganan outlier lainnya untuk mempertahankan ukuran data.")
        else:
            print("Dampak penghapusan SANGAT TINGGI (> 15%). Menghapus baris sangat tidak disarankan.")
            print("Winsorizing atau transformasi (misal, log transform) adalah pendekatan yang jauh lebih baik.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.4. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Harap jalankan SEL 3A.3 terlebih dahulu.")



## SEL 3A.4 (REVISI): Penghapusan Outlier Ekstrem (Berdasarkan 3 * IQR)

In [ ]:
# =============================================================================
# SEL 3A.4 (REVISI YANG BENAR): Penghapusan Outlier Ekstrem Terkontrol
# =============================================================================
print("--- SEL 3A.4: Memulai Penghapusan Baris dengan Outlier Ekstrem (3.0 * IQR) ---")

if 'df_cleaned' in globals():
    try:
        # --- 1. Pilih HANYA Fitur Inti untuk Diperiksa ---
        features_to_check = [
            'annual_inc', 'dti', 'revol_bal', 'open_acc', 'total_acc',
            'credit_history_length_years', 'loan_amnt', 'int_rate'
        ]
        features_to_check = [f for f in features_to_check if f in df_cleaned.columns]
        print(f"🎯 Fitur inti yang akan diperiksa untuk outlier: {features_to_check}")

        # --- 2. Dapatkan Indeks Outlier (menggunakan logika 3 * IQR) ---
        outlier_indices_to_drop = set()
        for col in features_to_check:
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR

            col_outliers = df_cleaned[
                (df_cleaned[col].notna()) &
                ((df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound))
            ].index
            outlier_indices_to_drop.update(col_outliers)

        # --- 3. Hapus Baris Outlier ---
        original_rows = len(df_cleaned)
        num_outliers = len(outlier_indices_to_drop)

        if num_outliers > 0:
            print(f"\n🗑️  Menghapus {num_outliers} baris yang mengandung outlier ekstrem...")
            df_cleaned.drop(index=list(outlier_indices_to_drop), inplace=True)

            rows_dropped = original_rows - len(df_cleaned)
            print(f"   -> Jumlah baris sebelum: {original_rows}")
            print(f"   -> Jumlah baris setelah: {len(df_cleaned)}")
            print(f"   -> Total baris yang dihapus: {rows_dropped} ({ (rows_dropped/original_rows)*100 :.2f}%)")
        else:
            print("\n✅ Tidak ditemukan outlier ekstrem untuk dihapus.")

        print("\n✅ SEL 3A.4 Selesai. Outlier ekstrem telah dihapus.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.4. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Jalankan SEL 3A.3 terlebih dahulu.")


In [ ]:
# =============================================================================
# SEL 3A.5 (REVISI): Final Checkpoint & Verification
# =============================================================================
print("--- SEL 3A.5: Verifikasi Akhir dan Penyimpanan Checkpoint ---")

if 'df_cleaned' in globals():
    # --- 1. Verifikasi Akhir dan Daftar Fitur Final ---
    print(f"\n📊 Verifikasi akhir DataFrame 'df_cleaned':")

    target_col = 'credit_risk'
    features = [col for col in df_cleaned.columns if col != target_col]

    print(f"   -> Dimensi Final: {df_cleaned.shape[0]} baris dan {df_cleaned.shape[1]} kolom.")
    print(f"   -> Jumlah Fitur Final: {len(features)}")

    numeric_features = df_cleaned[features].select_dtypes(include=np.number).columns.tolist()
    categorical_features = df_cleaned[features].select_dtypes(include='object').columns.tolist()

    print(f"\n   Fitur Numerik Final ({len(numeric_features)}):")
    print(numeric_features)

    print(f"\n   Fitur Kategorikal Final ({len(categorical_features)}):")
    print(categorical_features)

    # --- 2. Simpan Checkpoint Final ---
    if 'DATA_PROCESSED_DIR' in globals():
        final_checkpoint_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
        df_cleaned.to_csv(final_checkpoint_path, index=False)
        print(f"\n\n💾 Checkpoint final 'loan_data_model_ready.csv' berhasil disimpan di: {final_checkpoint_path}")
        print("   -> DataFrame ini sekarang siap untuk Bagian 3B (Train-Test Split & Pipeline).")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Jalankan sel-sel sebelumnya.")


## SEL 3A.4.1: Inspeksi Hasil Feature Engineering & Pembersihan

In [ ]:
# =============================================================================
# SEL 3A.4.1: Inspeksi Hasil Feature Engineering & Pembersihan
# =============================================================================
print("--- SEL 3A.1.1: Memulai Inspeksi Hasil ---")

if 'df_cleaned' in globals():
    try:
        # --- 1. Tampilkan 5 Baris Pertama dari Data yang Sudah Diproses ---
        print("\n--- Tampilan Awal DataFrame 'df_cleaned' ---")
        display(df_cleaned.head())

        # --- 2. Fokus pada Fitur-fitur Baru yang Dibuat ---
        new_features = [
            'credit_history_length_years',
            'loan_to_income_ratio',
            'installment_to_income_ratio',
            'revolving_util_ratio',
            'term_in_months',
            'credit_risk' # Juga target kita
        ]
        # Filter agar hanya menampilkan kolom yang ada
        new_features_in_df = [col for col in new_features if col in df_cleaned.columns]

        print("\n--- Inspeksi Statistik Deskriptif pada Fitur-fitur Baru ---")
        # .describe() akan memberikan gambaran tentang min, max, mean, dan juga jumlah non-null (count)
        display(df_cleaned[new_features_in_df].describe())

        # --- 3. Hitung dan Tampilkan Nilai NaN di Setiap Kolom ---
        print("\n--- Menghitung Jumlah dan Persentase Nilai NaN pada Data yang Diproses ---")

        nan_counts = df_cleaned.isnull().sum()
        nan_percentage = (nan_counts / len(df_cleaned)) * 100

        nan_info = pd.DataFrame({
            'Jumlah NaN': nan_counts,
            'Persentase NaN (%)': nan_percentage
        })

        # Tampilkan hanya kolom yang masih memiliki nilai NaN, diurutkan
        nan_info_filtered = nan_info[nan_info['Jumlah NaN'] > 0].sort_values(by='Jumlah NaN', ascending=False)

        if nan_info_filtered.empty:
            print("✅ Selamat! Tidak ada nilai NaN yang tersisa di dalam DataFrame.")
        else:
            print(f"Ditemukan {len(nan_info_filtered)} kolom yang masih mengandung nilai NaN:")
            display(nan_info_filtered)
            print("\nPenjelasan: Nilai-nilai NaN ini adalah normal dan akan ditangani pada tahap preprocessing selanjutnya (Bagian 3B) menggunakan imputer untuk mencegah data leakage.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3A.1.1. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'df_cleaned' tidak ditemukan. Harap jalankan SEL 3A.1 terlebih dahulu.")



# BAGIAN 3B: FINAL PREPROCESSING & TRAIN-TEST SPLIT

## SEL 3B.1: Pemuatan Data, Pemisahan Fitur-Target, dan Train-Test Split

In [ ]:
# =============================================================================
# SEL 3B.1 (REVISI): Pemuatan Data dan Train-Test Split
# =============================================================================
print("--- SEL 3B.1: Memulai Pemuatan Data dan Train-Test Split ---")

# Pastikan Anda sudah menjalankan Bagian 3A yang direvisi (SEL 3A.1, 3A.2, 3A.3)
# untuk menghasilkan file CSV yang benar.
try:
    file_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
    df_model_ready = pd.read_csv(file_path)
    print(f"✅ DataFrame 'loan_data_model_ready.csv' berhasil dimuat. Dimensi: {df_model_ready.shape}")

    X = df_model_ready.drop(columns=['credit_risk'])
    y = df_model_ready['credit_risk']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42, stratify=y
    )
    print("\n✅ Data berhasil di-split menjadi set latih dan uji.")
    print(f"   -> X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"   -> X_test: {X_test.shape}, y_test: {y_test.shape}")

except FileNotFoundError:
    print(f"❌ ERROR: File 'loan_data_model_ready.csv' tidak ditemukan. Harap jalankan Bagian 3A terlebih dahulu.")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 3B.1. Error: {e}")


## SEL 3B.2: Pembangunan Pipeline Preprocessing dengan ColumnTransformer

In [ ]:
# =============================================================================
# SEL 3B.2 (REVISI TOTAL): Pipeline Preprocessing dengan RobustScaler
# =============================================================================
print("\n--- SEL 3B.2: Membangun Pipeline Preprocessing yang Robust ---")

# Impor scaler baru
from sklearn.preprocessing import RobustScaler

if 'X_train' in globals():
    try:
        numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
        categorical_features = X_train.select_dtypes(include='object').columns.tolist()
        print(f"Ditemukan {len(numeric_features)} fitur numerik dan {len(categorical_features)} fitur kategorikal.")

        # --- Pipeline Numerik dengan RobustScaler ---
        numeric_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
            # GANTI StandardScaler dengan RobustScaler
            ('scaler', RobustScaler())
        ])
        print("\nPipeline untuk fitur numerik dibuat dengan RobustScaler (tahan terhadap outlier).")

        # --- Pipeline Kategorikal (tetap sama) ---
        categorical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])
        print("Pipeline untuk fitur kategorikal dibuat (Imputer + OneHotEncoder).")

        # --- Gabungkan dengan ColumnTransformer (tetap sama) ---
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_pipeline, numeric_features),
                ('cat', categorical_pipeline, categorical_features)
            ],
            remainder='passthrough'
        )
        print("\n✅ ColumnTransformer berhasil dibuat untuk menggabungkan semua pipeline.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3B.2. Error: {e}")
else:
    print("❌ ERROR: DataFrame 'X_train' tidak ditemukan. Jalankan SEL 3B.1 terlebih dahulu.")


## SEL 3B.3: Penerapan Pipeline dan Penyimpanan Data yang Telah Diproses

In [ ]:
# =============================================================================
# SEL 3B.3 (REVISI): Menerapkan Pipeline dan Menyimpan Hasil
# =============================================================================
print("\n--- SEL 3B.3: Menerapkan Pipeline dan Menyimpan Hasil ---")

if 'preprocessor' in globals():
    try:
        print("Menerapkan preprocessor pada X_train (fit_transform)...")
        X_train_processed = preprocessor.fit_transform(X_train)
        print(f"   -> X_train_processed dibuat dengan shape: {X_train_processed.shape}")

        print("\nMenerapkan preprocessor pada X_test (transform)...")
        X_test_processed = preprocessor.transform(X_test)
        print(f"   -> X_test_processed dibuat dengan shape: {X_test_processed.shape}")

        print("\n💾 Menyimpan semua data yang telah diproses dan preprocessor...")
        joblib.dump(preprocessor, os.path.join(MODELS_DIR, 'preprocessor.joblib'))
        np.save(os.path.join(DATA_PROCESSED_DIR, 'X_train_processed.npy'), X_train_processed)
        np.save(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'), X_test_processed)
        np.save(os.path.join(DATA_PROCESSED_DIR, 'y_train.npy'), y_train.values)
        np.save(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'), y_test.values)

        print("\n✅ Semua data telah siap untuk tahap pemodelan (Bagian 4).")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 3B.3. Error: {e}")
else:
    print("❌ ERROR: Object 'preprocessor' tidak ditemukan. Jalankan SEL 3B.2 terlebih dahulu.")


# Bagian 4: Eksperimen Komprehensif

## SEL 4.1: Pemuatan Data & Pengaturan Eksperimen

In [ ]:
# =============================================================================
# SEL 4.1: Pemuatan Data & Pengaturan Eksperimen
# =============================================================================
print("--- SEL 4.1: Memuat Data yang Telah Diproses dan Mengatur Eksperimen ---")

# Impor pustaka tambahan yang mungkin diperlukan untuk resampling
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline as ImbPipeline # Ganti nama agar tidak bentrok dengan pipeline sklearn
import time

if 'DATA_PROCESSED_DIR' in globals():
    try:
        # --- 1. Muat Semua Data yang Telah Diproses ---
        print("💾 Memuat data .npy dari disk...")
        X_train_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_train_processed.npy'))
        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        y_train = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_train.npy'))
        y_test = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'))
        print("   -> Semua data berhasil dimuat.")
        print(f"   -> Dimensi: X_train={X_train_processed.shape}, X_test={X_test_processed.shape}")

        # --- 2. Definisikan Model yang Akan Diuji ---
        # Kita akan menggunakan parameter dasar (default) untuk perbandingan awal.
        # random_state=42 digunakan untuk memastikan hasil yang dapat direproduksi.
        models = {
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
            'Random Forest': RandomForestClassifier(random_state=42, n_jobs=-1),
            'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1),
            'LightGBM': lgb.LGBMClassifier(random_state=42, n_jobs=-1),
            'CatBoost': cb.CatBoostClassifier(random_state=42, verbose=0) # verbose=0 agar tidak ada output saat training
        }
        print(f"\n🤖 {len(models)} model telah didefinisikan untuk diuji.")

        # --- 3. Definisikan Strategi Penanganan Data Tidak Seimbang ---
        # 'None' berarti tidak ada penanganan khusus.
        # 'Class Weight' akan ditangani di dalam loop.
        # SMOTE dan ADASYN akan diterapkan sebagai langkah resampling.
        resampling_strategies = {
            'None': None,
            'SMOTE': SMOTE(random_state=42),
            'ADASYN': ADASYN(random_state=42)
        }
        # Tambahkan 'Class Weight' sebagai penanda, bukan sebagai objek resampler
        all_strategies = ['None', 'Class Weight', 'SMOTE', 'ADASYN']
        print(f"⚖️  {len(all_strategies)} strategi penanganan data tidak seimbang akan diuji.")

        # --- 4. Siapkan Tempat untuk Menyimpan Hasil ---
        results_list = []
        print("\n✅ Pengaturan eksperimen selesai. Siap untuk memulai perbandingan model.")

    except FileNotFoundError:
        print(f"❌ ERROR: File .npy tidak ditemukan di {DATA_PROCESSED_DIR}. Harap jalankan Bagian 3B terlebih dahulu.")
    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 4.1. Error: {e}")
else:
    print("❌ ERROR: Variabel path 'DATA_PROCESSED_DIR' tidak ditemukan.")


## SEL 4.2: Eksekusi Eksperimen Perbandingan Model

In [ ]:
# =============================================================================
# SEL 4.2 (REVISI FINAL): Eksekusi Eksperimen dengan Metrik Tambahan & Penyimpanan
# =============================================================================
print("\n--- SEL 4.2: Memulai Eksekusi Eksperimen dengan Metrik Tambahan ---")

# Fungsi untuk menghitung statistik KS
def calculate_ks(y_true, y_pred_proba):
    """Menghitung statistik Kolmogorov-Smirnov."""
    df = pd.DataFrame({'y_true': y_true, 'y_pred_proba': y_pred_proba})
    df_sorted = df.sort_values(by='y_pred_proba', ascending=False)

    df_sorted['cum_bad'] = df_sorted['y_true'].cumsum() / df_sorted['y_true'].sum()
    df_sorted['cum_good'] = (1 - df_sorted['y_true']).cumsum() / (1 - df_sorted['y_true']).sum()

    ks_statistic = np.max(np.abs(df_sorted['cum_good'] - df_sorted['cum_bad']))

    return ks_statistic

if 'models' in globals():
    results_list = [] # Mulai ulang daftar hasil

    # Loop melalui setiap strategi
    for strategy_name in all_strategies:
        print(f"\n{'='*20} MENJALANKAN STRATEGI: {strategy_name.upper()} {'='*20}")

        # Loop melalui setiap model
        for model_name, model in models.items():
            start_time = time.time()

            X_train_resampled, y_train_resampled = X_train_processed, y_train

            # Logika untuk Class Weight dan Resampling
            if strategy_name == 'Class Weight':
                if hasattr(model, 'class_weight'): model.set_params(class_weight='balanced')
                elif hasattr(model, 'scale_pos_weight'):
                    scale = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
                    model.set_params(scale_pos_weight=scale)
            elif strategy_name in ['SMOTE', 'ADASYN']:
                print(f"   -> Menerapkan {strategy_name}...")
                resampler = resampling_strategies[strategy_name]
                X_train_resampled, y_train_resampled = resampler.fit_resample(X_train_processed, y_train)

            # Latih Model
            print(f"   -> Melatih model: {model_name}...")
            model.fit(X_train_resampled, y_train_resampled)

            # Lakukan Prediksi & Evaluasi
            y_pred = model.predict(X_test_processed)
            y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

            # Hitung Semua Metrik
            auc_score = roc_auc_score(y_test, y_pred_proba)
            ks_score = calculate_ks(y_test, y_pred_proba)
            gini_score = 2 * auc_score - 1

            metrics = {
                'Model': model_name,
                'Strategy': strategy_name,
                'AUC': auc_score,
                'Gini': gini_score,
                'KS': ks_score,
                'F1-Score': f1_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Accuracy': accuracy_score(y_test, y_pred),
                'Training Time (s)': time.time() - start_time
            }

            results_list.append(metrics)
            print(f"      -> Selesai. AUC: {metrics['AUC']:.4f}, Gini: {metrics['Gini']:.4f}, KS: {metrics['KS']:.4f}")

    # --- Konversi Hasil ke DataFrame ---
    results_df_comprehensive = pd.DataFrame(results_list)
    print("\n\n✅ SEMUA EKSPERIMEN SELESAI.")
    print("Hasil perbandingan model komprehensif tersimpan dalam DataFrame 'results_df_comprehensive'.")

    # =========================================================================
    # --- BLOK BARU: Simpan Hasil ke File CSV ---
    # =========================================================================
    if 'RESULTS_DIR' in globals():
        try:
            # Buat subfolder jika belum ada
            analysis_data_dir = os.path.join(RESULTS_DIR, 'analysis_data')
            os.makedirs(analysis_data_dir, exist_ok=True)

            # Tentukan path file
            results_file_path = os.path.join(analysis_data_dir, 'model_comparison_results.csv')

            # Simpan DataFrame ke CSV
            results_df_comprehensive.to_csv(results_file_path, index=False)

            print(f"\n💾 Hasil eksperimen berhasil disimpan ke: {results_file_path}")

        except Exception as e:
            print(f"\n❌ GAGAL menyimpan hasil eksperimen. Error: {e}")
    # =========================================================================

else:
    print("❌ ERROR: Objek 'models' tidak ditemukan. Jalankan SEL 4.1 terlebih dahulu.")


## SEL 4.3 (BARU): Analisis Tabel Perbandingan Komprehensif

In [ ]:
# =============================================================================
# SEL 4.3 (BARU): Analisis Tabel Perbandingan Komprehensif
# =============================================================================
print("--- SEL 4.3: Menganalisis Hasil Perbandingan Model Komprehensif ---")

if 'results_df_comprehensive' in globals():
    # --- 1. Tampilkan Tabel Hasil Lengkap ---
    # Urutkan berdasarkan metrik yang paling penting untuk pemisahan, misal Gini atau KS.
    print("\n--- Tabel Perbandingan Model (Diurutkan berdasarkan Gini) ---")
    display(results_df_comprehensive.sort_values(by='Gini', ascending=False))

    print("\n--- Tabel Perbandingan Model (Diurutkan berdasarkan KS) ---")
    display(results_df_comprehensive.sort_values(by='KS', ascending=False))

    # --- 2. Visualisasi Perbandingan Metrik Kunci ---
    # Fokus pada metrik pemisahan (AUC, Gini, KS)
    metrics_to_plot = ['AUC', 'Gini', 'KS']

    # Ambil 10 kombinasi model/strategi terbaik berdasarkan AUC
    top_10_results = results_df_comprehensive.sort_values(by='AUC', ascending=False).head(10)
    top_10_results['Model_Strategy'] = top_10_results['Model'] + ' (' + top_10_results['Strategy'] + ')'

    plt.figure(figsize=(12, 8))
    # Lelehkan dataframe untuk format yang ramah untuk seaborn
    df_melted = top_10_results.melt(
        id_vars='Model_Strategy',
        value_vars=metrics_to_plot,
        var_name='Metric',
        value_name='Score'
    )

    sns.barplot(data=df_melted, x='Score', y='Model_Strategy', hue='Metric', palette='viridis')
    plt.title('Perbandingan Metrik Pemisahan untuk 10 Model Terbaik')
    plt.xlabel('Score')
    plt.ylabel('Model (Strategy)')
    plt.legend(title='Metric')
    plt.grid(axis='x')
    plt.tight_layout()
    plt.show()

else:
    print("❌ ERROR: DataFrame 'results_df_comprehensive' tidak ditemukan. Jalankan SEL 4.2 yang direvisi terlebih dahulu.")


## SEL 4.4: Investigasi Threshold Optimal (Multi-Strategi)

In [ ]:
# =============================================================================
# SEL 4.4: Investigasi Threshold Optimal (Multi-Strategi)
# =============================================================================
print("\n--- SEL 4.2.1+: Investigasi Threshold Optimal (Multi-Strategi) ---")

from sklearn.metrics import (
    f1_score, recall_score, precision_score,
    precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# --- 1. Pilih Model & Strategi Terbaik ---
best_model_name = 'XGBoost'
best_strategy_name = 'Class Weight'
print(f"Model terbaik: {best_model_name} | Strategi: {best_strategy_name}")

# --- 2. Buat Validation Split dari Training ---
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train_processed, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# --- 3. Latih Model pada Train_sub ---
scale = len(y_train_sub[y_train_sub == 0]) / len(y_train_sub[y_train_sub == 1])
model_to_tune = xgb.XGBClassifier(
    random_state=42, use_label_encoder=False, eval_metric='logloss',
    n_jobs=-1, scale_pos_weight=scale
)

print("Melatih ulang model terbaik pada subset train...")
model_to_tune.fit(X_train_sub, y_train_sub)

# --- 4. Prediksi Probabilitas ---
y_val_proba = model_to_tune.predict_proba(X_val)[:, 1]
y_test_proba = model_to_tune.predict_proba(X_test_processed)[:, 1]

# ========================================================================
# 🔹 Metode 1: Grid Search F1 (manual loop)
# ========================================================================
thresholds = np.arange(0.01, 1.0, 0.01)
f1_scores = [f1_score(y_val, (y_val_proba >= t).astype(int)) for t in thresholds]

opt_f1_idx = np.argmax(f1_scores)
opt_f1_th = thresholds[opt_f1_idx]

# ========================================================================
# 🔹 Metode 2: Dari Precision-Recall Curve
# ========================================================================
precisions, recalls, ths_pr = precision_recall_curve(y_val, y_val_proba)
f1_scores_pr = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
opt_pr_idx = np.nanargmax(f1_scores_pr)
opt_pr_th = ths_pr[opt_pr_idx]

# ========================================================================
# 🔹 Metode 3: Dari ROC Curve (Youden’s J = TPR - FPR)
# ========================================================================
fpr, tpr, ths_roc = roc_curve(y_val, y_val_proba)
j_scores = tpr - fpr
opt_roc_idx = np.argmax(j_scores)
opt_roc_th = ths_roc[opt_roc_idx]

# ========================================================================
# 🔹 Evaluasi semua threshold di Test Set
# ========================================================================
def evaluate_preds(y_true, y_proba, threshold, label=""):
    y_pred = (y_proba >= threshold).astype(int)
    return {
        "Label": label,
        "Threshold": threshold,
        "F1": f1_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba)
    }

results_compare = pd.DataFrame([
    evaluate_preds(y_test, y_test_proba, 0.5, "Default (0.5)"),
    evaluate_preds(y_test, y_test_proba, opt_f1_th, f"F1-Grid ({opt_f1_th:.2f})"),
    evaluate_preds(y_test, y_test_proba, opt_pr_th, f"PR-Curve ({opt_pr_th:.2f})"),
    evaluate_preds(y_test, y_test_proba, opt_roc_th, f"ROC-Youden ({opt_roc_th:.2f})"),
])

display(results_compare)

# ========================================================================
# 🔹 Visualisasi Threshold vs F1
# ========================================================================
plt.figure(figsize=(10, 5))
plt.plot(thresholds, f1_scores, marker='o', linestyle='--', label="Grid F1")
plt.axvline(opt_f1_th, color='r', linestyle='--', label=f"F1 Opt = {opt_f1_th:.2f}")
plt.axvline(opt_pr_th, color='g', linestyle='--', label=f"PR Opt = {opt_pr_th:.2f}")
plt.axvline(opt_roc_th, color='b', linestyle='--', label=f"ROC Opt = {opt_roc_th:.2f}")
plt.title("Threshold vs F1-Score (Validation Set)")
plt.xlabel("Threshold")
plt.ylabel("F1-Score")
plt.legend()
plt.grid(True)
plt.show()

# ========================================================================
# 🔹 Precision-Recall Curve
# ========================================================================
plt.figure(figsize=(7, 5))
plt.plot(recalls, precisions, label="Precision-Recall Curve")
plt.axvline(x=recall_score(y_test, (y_test_proba >= opt_pr_th).astype(int)),
            color="g", linestyle="--", label=f"PR Opt Th={opt_pr_th:.2f}")
plt.title("Precision-Recall Curve (Test Set)")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.grid(True)
plt.show()


## SEL 4.5: Hyperparameter Tuning untuk Model-model Terbaik

In [ ]:
# =============================================================================
# SEL 4.5 (FINAL & ROBUST): Hyperparameter Tuning & Penyimpanan Semua Model
# =============================================================================
print("--- SEL 4.5 (FINAL): Memulai Hyperparameter Tuning & Penyimpanan Semua Model ---")

# Impor pustaka yang diperlukan
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import time
import json # Untuk menyimpan parameter dalam format yang rapi

# Pastikan data yang telah diproses sudah ada
if 'X_train_processed' in globals():

    # --- 1. Definisikan Ruang Parameter yang Dioptimalkan ---
    param_grids = {
        'Logistic Regression': {
            'C': uniform(0.01, 10),
            'solver': ['liblinear']
        },
        'LightGBM': {
            'n_estimators': randint(100, 800),
            'learning_rate': uniform(0.01, 0.1),
            'num_leaves': randint(20, 60),
            'max_depth': [-1, 5, 10, 15],
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4)
        },
        'XGBoost': {
            'n_estimators': randint(100, 800),
            'learning_rate': uniform(0.01, 0.1),
            'max_depth': randint(3, 10),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'gamma': [0, 1, 5]
        },
        'CatBoost': {
            'iterations': randint(100, 800),
            'learning_rate': uniform(0.01, 0.1),
            'depth': randint(4, 10),
            'l2_leaf_reg': uniform(1, 10)
        }
    }

    # --- 2. Siapkan Model ---
    scale = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    models_to_tune = {
        #'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
        #'LightGBM': lgb.LGBMClassifier(random_state=42, n_jobs=-1, class_weight='balanced'),
        #'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1, scale_pos_weight=scale),
        'CatBoost': cb.CatBoostClassifier(random_state=42, verbose=0, auto_class_weights='Balanced')
    }

    # --- 3. Jalankan Proses Tuning ---
    tuning_results = []
    best_estimators = {} # Dictionary untuk menyimpan model terbaik dari setiap jenis

    for model_name, model in models_to_tune.items():
        print(f"\n{'='*25}\n🚀 TUNING MODEL: {model_name}\n{'='*25}")

        random_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grids[model_name],
            n_iter=25,
            cv=3,
            scoring='f1',
            n_jobs=-1,
            random_state=42,
            verbose=1
        )

        start_time = time.time()
        print("Memulai Randomized Search...")
        random_search.fit(X_train_processed, y_train)
        end_time = time.time()

        # Simpan hasil tuning
        tuning_results.append({
            'Model': model_name,
            'Best Score (CV F1)': random_search.best_score_,
            'Best Params': random_search.best_params_,
            'Tuning Time (m)': (end_time - start_time) / 60
        })

        # Simpan model terbaik ke dalam dictionary
        best_estimators[model_name] = random_search.best_estimator_

        print(f"✅ Tuning untuk {model_name} selesai dalam {(end_time - start_time) / 60:.2f} menit.")
        print(f"   -> Skor F1 CV Terbaik: {random_search.best_score_:.4f}")

    # --- 4. Tampilkan Ringkasan Hasil Tuning ---
    print("\n\n" + "="*60)
    print("      RINGKASAN HASIL HYPERPARAMETER TUNING")
    print("="*60)
    tuning_results_df = pd.DataFrame(tuning_results)
    display(tuning_results_df.sort_values(by='Best Score (CV F1)', ascending=False))

    # =========================================================================
    # --- BAGIAN BARU: Simpan SEMUA Model yang Telah Di-tuning ---
    # =========================================================================
    print("\n" + "="*60)
    print("      💾 MENYIMPAN SEMUA MODEL TERBAIK")
    print("="*60)

    if 'MODELS_DIR' in globals():
        # Loop melalui dictionary 'best_estimators' yang sudah kita isi
        for model_name, model_object in best_estimators.items():
            # Buat nama file yang bersih (misal, 'Logistic Regression' -> 'Logistic_Regression')
            safe_model_name = model_name.replace(" ", "_")
            model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

            # Simpan model
            joblib.dump(model_object, model_path)
            print(f"   -> ✅ Model '{model_name}' berhasil disimpan di: {model_path}")

        # Juga simpan DataFrame hasil tuning untuk referensi nanti
        tuning_results_df.to_csv(os.path.join(RESULTS_DIR, 'analysis_data', 'tuning_summary.csv'), index=False)
        print("\n   -> ✅ Ringkasan hasil tuning juga disimpan dalam file CSV.")

    print("\nSemua model telah disimpan. Anda bisa me-restart sesi tanpa kehilangan hasil tuning.")
    print("="*60)
    # =========================================================================

    # --- 5. Evaluasi Cepat Model Juara (Opsional, karena sudah disimpan) ---
    champion_model_name = tuning_results_df.sort_values(by='Best Score (CV F1)', ascending=False).iloc[0]['Model']
    champion_model = best_estimators[champion_model_name]

    print(f"\n\n--- Laporan Klasifikasi Cepat untuk Model Juara ({champion_model_name}) pada Data Uji ---")
    y_pred_test = champion_model.predict(X_test_processed)
    print(classification_report(y_test, y_pred_test))

    cm = confusion_matrix(y_test, y_pred_test)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['Good', 'Bad'], yticklabels=['Good', 'Bad'])
    plt.title(f'Confusion Matrix - {champion_model_name} (Tuned)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

else:
    print("❌ ERROR: Data yang telah diproses tidak ditemukan. Jalankan Bagian 3B terlebih dahulu.")



In [ ]:
# =============================================================================
# SEL 4.5: Perbandingan Confusion Matrix untuk Model yang Telah Di-tuning
# =============================================================================
print("--- SEL 4.5: Memulai Perbandingan Confusion Matrix ---")

# Pastikan hasil tuning sudah ada
if 'best_estimators' in globals():
    try:
        # Tentukan jumlah baris dan kolom untuk subplot
        n_models = len(best_estimators)
        n_cols = 2
        n_rows = (n_models + 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 5))
        axes = axes.flatten() # Ubah menjadi array 1D agar mudah di-loop

        # Loop melalui setiap model terbaik
        for i, (model_name, model) in enumerate(best_estimators.items()):
            print(f"   -> Membuat Confusion Matrix untuk: {model_name}")

            # Lakukan prediksi pada data uji
            y_pred = model.predict(X_test_processed)

            # Buat confusion matrix
            cm = confusion_matrix(y_test, y_pred)

            # Plot heatmap
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                        xticklabels=['Good Loan', 'Bad Loan'],
                        yticklabels=['Good Loan', 'Bad Loan'])

            axes[i].set_title(f'Confusion Matrix - {model_name}')
            axes[i].set_xlabel('Predicted Label')
            axes[i].set_ylabel('True Label')

        # Sembunyikan sumbu yang tidak digunakan jika jumlah model ganjil
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 4.5. Error: {e}")
else:
    print("❌ ERROR: Objek 'best_estimators' tidak ditemukan. Jalankan SEL 4.4 terlebih dahulu.")





## SEL 4.6: Perbandingan Learning Curve untuk Model yang Telah Di-tuning

In [ ]:
# =============================================================================
# SEL 4.6: Perbandingan Learning Curve untuk Model yang Telah Di-tuning
# =============================================================================
print("\n--- SEL 4.6: Memulai Perbandingan Learning Curve ---")

# Impor pustaka yang diperlukan
from sklearn.model_selection import learning_curve

# Pastikan hasil tuning sudah ada
if 'best_estimators' in globals():
    try:
        # Tentukan jumlah baris dan kolom untuk subplot
        n_models = len(best_estimators)
        n_cols = 2
        n_rows = (n_models + 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 8, n_rows * 6), sharey=True)
        axes = axes.flatten()

        # Loop melalui setiap model terbaik
        for i, (model_name, model) in enumerate(best_estimators.items()):
            print(f"   -> Membuat Learning Curve untuk: {model_name}...")

            # Hasilkan data untuk learning curve
            # cv=3: menggunakan 3-fold cross-validation
            # scoring='f1': metrik yang kita pedulikan
            # n_jobs=-1: gunakan semua core CPU
            train_sizes, train_scores, validation_scores = learning_curve(
                estimator=model,
                X=X_train_processed,
                y=y_train,
                cv=3,
                scoring='f1',
                n_jobs=-1,
                train_sizes=np.linspace(0.1, 1.0, 10), # Gunakan 10 titik ukuran data
                random_state=42
            )

            # Hitung rata-rata dan standar deviasi
            train_scores_mean = np.mean(train_scores, axis=1)
            train_scores_std = np.std(train_scores, axis=1)
            validation_scores_mean = np.mean(validation_scores, axis=1)
            validation_scores_std = np.std(validation_scores, axis=1)

            # Plot kurva
            ax = axes[i]
            ax.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training score')
            ax.plot(train_sizes, validation_scores_mean, 'o-', color='green', label='Cross-validation score')

            # Plot area standar deviasi
            ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                            train_scores_mean + train_scores_std, alpha=0.1, color='blue')
            ax.fill_between(train_sizes, validation_scores_mean - validation_scores_std,
                            validation_scores_mean + validation_scores_std, alpha=0.1, color='green')

            ax.set_title(f'Learning Curve - {model_name}')
            ax.set_xlabel('Training examples')
            ax.set_ylabel('F1 Score')
            ax.legend(loc='best')
            ax.grid(True)

        # Sembunyikan sumbu yang tidak digunakan
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 4.6. Error: {e}")
else:
    print("❌ ERROR: Objek 'best_estimators' tidak ditemukan. Jalankan SEL 4.4 terlebih dahulu.")

## SEL 4.7 (REVISI LENGKAP): Evaluasi Metrik Komprehensif dari Model Tuned

In [ ]:
# =============================================================================
# SEL 4.7 (REVISI LENGKAP): Evaluasi Metrik Komprehensif dari Model Tuned
# =============================================================================
print("--- SEL 4.7: Memulai Evaluasi Metrik Lengkap dari Semua Model yang Telah Di-tuning ---")

# --- Fungsi Helper untuk Menghitung KS ---
def calculate_ks(y_true, y_pred_proba):
    """Menghitung statistik Kolmogorov-Smirnov."""
    df = pd.DataFrame({'y_true': y_true, 'y_pred_proba': y_pred_proba})
    df_sorted = df.sort_values(by='y_pred_proba', ascending=False)

    # Hitung distribusi kumulatif untuk kelas 'good' (0) dan 'bad' (1)
    df_sorted['cum_bad'] = df_sorted['y_true'].cumsum() / df_sorted['y_true'].sum()
    df_sorted['cum_good'] = (1 - df_sorted['y_true']).cumsum() / (1 - df_sorted['y_true']).sum()

    # KS adalah perbedaan maksimum antara dua distribusi kumulatif
    ks_statistic = np.max(np.abs(df_sorted['cum_good'] - df_sorted['cum_bad']))

    return ks_statistic

try:
    # --- 1. Pastikan Data Uji Tersedia ---
    if 'X_test_processed' not in globals() or 'y_test' not in globals():
        print("⚠️  Data uji tidak ditemukan di memori, memuat ulang dari file...")
        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        y_test = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'))
        print("   -> Data uji berhasil dimuat.")

    # --- 2. Loop Melalui Semua Model Tersimpan ---
    model_names = ['Logistic Regression', 'LightGBM', 'XGBoost', 'CatBoost']
    evaluation_results = []

    print("\n📊 Mengevaluasi setiap model yang telah di-tuning pada data uji...")
    for model_name in model_names:
        safe_model_name = model_name.replace(" ", "_")
        # Sesuaikan path untuk CatBoost jika namanya berbeda
        if model_name == 'CatBoost' and not os.path.exists(os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')):
             model_path = os.path.join(MODELS_DIR, f'champion_model_{safe_model_name}.joblib')
        else:
             model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

        try:
            # Muat model
            model = joblib.load(model_path)

            # Lakukan prediksi (ini cepat)
            y_pred = model.predict(X_test_processed)
            y_pred_proba = model.predict_proba(X_test_processed)[:, 1]

            # --- 3. Hitung SEMUA Metrik ---
            auc_score = roc_auc_score(y_test, y_pred_proba)

            metrics = {
                'Model': model_name,
                'AUC': auc_score,
                'Gini': 2 * auc_score - 1, # Hitung Gini dari AUC
                'KS': calculate_ks(y_test, y_pred_proba), # Hitung KS
                'F1-Score': f1_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Accuracy': accuracy_score(y_test, y_pred)
            }
            evaluation_results.append(metrics)
            print(f"   -> Metrik untuk '{model_name}' berhasil dihitung.")

        except FileNotFoundError:
            print(f"   -> ❌ PERINGATAN: File model untuk '{model_name}' tidak ditemukan. Model ini akan dilewati.")
            continue

    # --- 4. Tampilkan Hasil dalam Tabel Perbandingan ---
    if not evaluation_results:
        print("\n❌ ERROR: Tidak ada model yang berhasil dievaluasi.")
    else:
        df_metrics = pd.DataFrame(evaluation_results)

        # Tampilkan dua tabel: satu diurutkan berdasarkan Gini, satu berdasarkan F1-Score
        print("\n" + "="*80)
        print("      TABEL PERBANDINGAN METRIK (Diurutkan berdasarkan Gini)")
        print("="*80)
        display(df_metrics.sort_values(by='Gini', ascending=False))

        print("\n" + "="*80)
        print("      TABEL PERBANDINGAN METRIK (Diurutkan berdasarkan F1-Score)")
        print("="*80)
        display(df_metrics.sort_values(by='F1-Score', ascending=False))
        print("="*80)

except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.7. Error: {e}")



## SEL 4.8 (BARU): Visualisasi Perbandingan Confusion Matrix Multi-Model

In [ ]:
# =============================================================================
# SEL 4.8 (BARU): Visualisasi Perbandingan Confusion Matrix Multi-Model
# =============================================================================
print("--- SEL 4.8: Memulai Visualisasi Perbandingan Confusion Matrix ---")

try:
    # Pastikan data uji sudah ada di memori
    if 'X_test_processed' not in globals() or 'y_test' not in globals():
        print("⚠️  Data uji tidak ditemukan, memuat ulang dari file...")
        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        y_test = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'))
        print("   -> Data uji berhasil dimuat.")

    # --- 1. Siapkan Model dan Subplot ---
    model_names = ['Logistic Regression', 'LightGBM', 'XGBoost', 'CatBoost']

    # Buat layout subplot 2x2
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    # 'Flatten' array axes agar mudah di-loop (dari 2D menjadi 1D)
    axes = axes.flatten()

    print("\n🖼️  Membuat plot Confusion Matrix untuk setiap model...")

    # --- 2. Loop, Muat Model, dan Buat Plot ---
    for i, model_name in enumerate(model_names):
        safe_model_name = model_name.replace(" ", "_")
        # Sesuaikan path untuk CatBoost jika namanya berbeda
        if model_name == 'CatBoost' and not os.path.exists(os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')):
             model_path = os.path.join(MODELS_DIR, f'champion_model_{safe_model_name}.joblib')
        else:
             model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

        try:
            # Muat model
            model = joblib.load(model_path)

            # Buat prediksi
            y_pred = model.predict(X_test_processed)

            # Hitung confusion matrix
            cm = confusion_matrix(y_test, y_pred)

            # --- Buat Plot Heatmap ---
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                        xticklabels=['Good Loan', 'Bad Loan'],
                        yticklabels=['Good Loan', 'Bad Loan'])

            axes[i].set_title(f'Confusion Matrix - {model_name}', fontsize=14)
            axes[i].set_xlabel('Predicted Label', fontsize=12)
            axes[i].set_ylabel('True Label', fontsize=12)
            print(f"   -> Plot untuk '{model_name}' berhasil dibuat.")

        except FileNotFoundError:
            print(f"   -> ❌ PERINGATAN: File model untuk '{model_name}' tidak ditemukan. Plot ini akan kosong.")
            axes[i].set_title(f'Model {model_name} Tidak Ditemukan')
            axes[i].axis('off') # Sembunyikan sumbu jika model tidak ada
            continue

    # --- 3. Tampilkan Plot ---
    plt.tight_layout(pad=3.0) # Beri sedikit ruang antar plot
    plt.show()

except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.8. Error: {e}")



## SEL 4.9 (BARU): Perbandingan Laporan Klasifikasi Multi-Model

In [ ]:
# =============================================================================
# SEL 4.9 (BARU): Perbandingan Laporan Klasifikasi Multi-Model
# =============================================================================
print("--- SEL 4.9: Memulai Perbandingan Laporan Klasifikasi untuk Semua Model ---")

try:
    # Pastikan data uji sudah ada di memori
    if 'X_test_processed' not in globals() or 'y_test' not in globals():
        print("⚠️  Data uji tidak ditemukan, memuat ulang dari file...")
        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        y_test = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'))
        print("   -> Data uji berhasil dimuat.")

    # --- 1. Siapkan Model ---
    model_names = ['Logistic Regression', 'LightGBM', 'XGBoost', 'CatBoost']

    print("\n" + "="*65)
    print("      LAPORAN KLASIFIKASI UNTUK SETIAP MODEL (PADA DATA UJI)")
    print("="*65)

    # --- 2. Loop, Muat Model, dan Tampilkan Laporan ---
    for model_name in model_names:
        safe_model_name = model_name.replace(" ", "_")
        # Sesuaikan path untuk CatBoost jika namanya berbeda
        if model_name == 'CatBoost' and not os.path.exists(os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')):
             model_path = os.path.join(MODELS_DIR, f'champion_model_{safe_model_name}.joblib')
        else:
             model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

        try:
            # Muat model
            model = joblib.load(model_path)

            # Buat prediksi
            y_pred = model.predict(X_test_processed)

            # --- Cetak Laporan Klasifikasi ---
            print(f"\n\n--- Laporan untuk Model: {model_name} ---")

            # Gunakan target_names untuk label yang lebih deskriptif
            report = classification_report(y_test, y_pred, target_names=['Good Loan (0)', 'Bad Loan (1)'])
            print(report)
            print("-" * 65)

        except FileNotFoundError:
            print(f"\n\n--- Laporan untuk Model: {model_name} ---")
            print(f"   -> ❌ PERINGATAN: File model tidak ditemukan. Laporan tidak dapat dibuat.")
            print("-" * 65)
            continue

    print("\n✅ Semua laporan klasifikasi telah ditampilkan.")

except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.9. Error: {e}")



# Bagian 5: Interpretasi Model


## SEL 5.1 (DINAMIS): Pemuatan Model Pilihan & Interpretasi Global SHAP

In [ ]:
# =============================================================================
# SEL 5.1 (DINAMIS): Pemuatan Model Pilihan & Interpretasi Global SHAP
# =============================================================================
# --- UBAH VARIABEL INI UNTUK MENGANALISIS MODEL YANG BERBEDA ---
# Pilihan yang tersedia: 'Logistic Regression', 'LightGBM', 'XGBoost', 'CatBoost'

MODEL_TO_ANALYZE = 'CatBoost'

# ---------------------------------------------------------------

print(f"✅ Konfigurasi diatur untuk menganalisis model: '{MODEL_TO_ANALYZE}'")



print(f"\n--- SEL 5.1: Memuat '{MODEL_TO_ANALYZE}' dan Memulai Interpretasi Global SHAP ---")

try:
    # --- 1. Tentukan dan Muat Model yang Dipilih ---
    safe_model_name = MODEL_TO_ANALYZE.replace(" ", "_")
    model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

    print(f"💾 Memuat model '{MODEL_TO_ANALYZE}' dari: {model_path}")
    # Kita gunakan nama variabel 'selected_model' agar lebih jelas
    selected_model = joblib.load(model_path)
    print("   -> Model berhasil dimuat.")

    # --- 2. Muat Preprocessor dan Data Uji ---
    preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
    preprocessor = joblib.load(preprocessor_path)
    print("   -> Preprocessor berhasil dimuat.")

    if 'X_test_processed' not in globals():
        print("   -> Data uji tidak ditemukan, memuat ulang...")
        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        y_test = np.load(os.path.join(DATA_PROCESSED_DIR, 'y_test.npy'))

    # --- 3. Buat SHAP Explainer ---
    print(f"\nMembuat SHAP Explainer untuk model {MODEL_TO_ANALYZE}...")
    # Gunakan try-except untuk menangani model linear vs. tree
    try:
        # Untuk model berbasis pohon (XGBoost, LightGBM, CatBoost)
        explainer = shap.TreeExplainer(selected_model)
    except Exception:
        # Untuk model linear (Logistic Regression)
        # SHAP membutuhkan data latih untuk model linear
        X_train_processed_sample = shap.sample(X_train_processed, 200, random_state=42)
        explainer = shap.KernelExplainer(selected_model.predict_proba, X_train_processed_sample)
        print("   -> Menggunakan KernelExplainer untuk model non-tree.")

    # --- 4. Hitung Nilai SHAP ---
    print("Menghitung nilai SHAP pada sampel data uji...")
    X_test_sample = shap.sample(X_test_processed, 1000, random_state=42) # Sampel lebih kecil untuk kecepatan

    # Penanganan berbeda untuk TreeExplainer vs KernelExplainer
    if isinstance(explainer, shap.explainers.Tree):
        shap_values = explainer.shap_values(X_test_sample)
    else: # KernelExplainer
        shap_values = explainer.shap_values(X_test_sample, l1_reg="aic")[1] # Ambil untuk kelas positif (1)

    # --- 5. Dapatkan Nama Fitur ---
    print("Mendapatkan nama fitur yang telah diproses...")
    num_features_imputed = preprocessor.transformers_[0][1].get_feature_names_out()
    cat_features_encoded = preprocessor.transformers_[1][1].get_feature_names_out()
    processed_feature_names = np.concatenate([num_features_imputed, cat_features_encoded])

    # --- 6. Buat SHAP Summary Plot ---
    print(f"\n--- SHAP Summary Plot untuk Model {MODEL_TO_ANALYZE} ---")
    X_test_sample_df = pd.DataFrame(X_test_sample, columns=processed_feature_names)
    shap.summary_plot(shap_values, X_test_sample_df, plot_type="dot")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: File model atau preprocessor tidak ditemukan. Pastikan SEL 4.4 sudah dijalankan.")
    print(f"   Detail Error: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 5.1. Error: {e}")



## SEL 5.2 (REVISI FINAL & ROBUST): Interpretasi Lokal untuk Model Pilihan

In [ ]:
# =============================================================================
# SEL 5.2 (REVISI FINAL & ROBUST): Interpretasi Lokal untuk Model Pilihan
# =============================================================================
print(f"\n\n--- SEL 5.2 (REVISI): Memulai Interpretasi Lokal untuk Model {MODEL_TO_ANALYZE} ---")

# Pastikan semua variabel dari SEL 5.1 sudah ada
if 'explainer' in locals() and 'selected_model' in locals():
    try:
        shap.initjs()

        safe_model_name_lower = MODEL_TO_ANALYZE.replace(" ", "_").lower()
        VIZ_DIR_SHAP = os.path.join(RESULTS_DIR, f"visualizations/shap_{safe_model_name_lower}")
        os.makedirs(VIZ_DIR_SHAP, exist_ok=True)
        print(f"🖼️  Plot SHAP lokal untuk {MODEL_TO_ANALYZE} akan disimpan di: {VIZ_DIR_SHAP}")

        y_pred_test_full = selected_model.predict(X_test_processed)
        tp_indices = np.where((y_test == 1) & (y_pred_test_full == 1))[0]
        tn_indices = np.where((y_test == 0) & (y_pred_test_full == 0))[0]

        if len(tp_indices) > 0 and len(tn_indices) > 0:
            idx_bad_loan = tp_indices[0]
            idx_good_loan = tn_indices[0]

            # ====================================================================
            # --- FUNGSI HELPER YANG SUDAH DIPERBAIKI ---
            # ====================================================================
            def create_and_save_force_plot(shap_values_single, features_single, idx, plot_type):
                """Fungsi untuk membuat, menyimpan, dan menampilkan force plot yang bersih dan adaptif."""
                print(f"\n--- Menjelaskan Plot untuk Nasabah {plot_type.title()} (Indeks Uji: {idx}) ---")

                # --- BLOK KODE PERBAIKAN ---
                # Cek apakah output SHAP adalah list (punya 2 elemen) atau hanya satu array
                expected_value = explainer.expected_value

                # Jika expected_value adalah list/array, ambil nilai untuk kelas positif (1)
                if isinstance(expected_value, (list, np.ndarray)) and len(np.array(expected_value).shape) > 0:
                    expected_value = expected_value[1]

                # Jika shap_values_single adalah list/array dari list, ambil nilai untuk kelas positif (1)
                # Ini adalah cara aman untuk menangani output [shap_class_0, shap_class_1]
                if isinstance(shap_values_single, list) and len(shap_values_single) == 2:
                    shap_values_for_plot = shap_values_single[1]
                else:
                    # Jika bukan list, berarti sudah merupakan array yang benar
                    shap_values_for_plot = shap_values_single
                # --- AKHIR BLOK PERBAIKAN ---

                # Buat plot versi gambar (matplotlib)
                plt.figure()
                shap.force_plot(expected_value, shap_values_for_plot, features_single,
                                feature_names=processed_feature_names, matplotlib=True, show=False, contribution_threshold=0.05)
                file_path = os.path.join(VIZ_DIR_SHAP, f"shap_force_plot_{plot_type}_loan.png")
                plt.savefig(file_path, dpi=150, bbox_inches='tight')
                plt.close()
                print(f"   -> Versi gambar disimpan ke: {file_path}")

                # Tampilkan plot versi interaktif (javascript)
                print("   -> Menampilkan versi interaktif:")
                display(shap.force_plot(expected_value, shap_values_for_plot, features=features_single, feature_names=processed_feature_names))

            # --- Proses Penjelasan (tidak ada perubahan di sini) ---
            # Proses untuk nasabah berisiko
            row_bad_loan_numpy = X_test_processed[idx_bad_loan:idx_bad_loan+1, :]
            shap_values_bad = explainer.shap_values(row_bad_loan_numpy)
            create_and_save_force_plot(shap_values_bad, row_bad_loan_numpy, idx_bad_loan, "bad")

            # Proses untuk nasabah aman
            row_good_loan_numpy = X_test_processed[idx_good_loan:idx_good_loan+1, :]
            shap_values_good = explainer.shap_values(row_good_loan_numpy)
            create_and_save_force_plot(shap_values_good, row_good_loan_numpy, idx_good_loan, "good")

        else:
            print(f"Tidak dapat menemukan contoh True Positive atau True Negative untuk dijelaskan dengan model {MODEL_TO_ANALYZE}.")

    except Exception as e:
        print(f"\n❌ GAGAL pada SEL 5.2. Error: {e}")
else:
    print("❌ ERROR: Variabel dari SEL 5.1 tidak ditemukan. Jalankan SEL 5.1 terlebih dahulu.")

## SEL 5.3 (MANDIRI): Pemuatan Artefak & Analisis Dampak Finansial

In [ ]:
# =============================================================================
# SEL 5.3 (MANDIRI): Pemuatan Artefak & Analisis Dampak Finansial
# =============================================================================
print("--- SEL 5.3 (MANDIRI): Memuat Artefak dan Memulai Analisis Finansial ---")

try:
    # --- BAGIAN 0: PEMUATAN SEMUA KOMPONEN YANG DIPERLUKAN ---
    print("💾 Memuat semua artefak yang diperlukan dari disk...")

    # 1. Tentukan path ke model, preprocessor, dan data
    model_path = "/content/drive/MyDrive/Colab_Projects/Credit_Risk_Prediction#1/02_models/champion_model_CatBoost.joblib"
    preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
    data_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")

    # 2. Muat model dan preprocessor
    champion_model = joblib.load(model_path)
    preprocessor = joblib.load(preprocessor_path)
    print(f"   -> Model '{champion_model.__class__.__name__}' berhasil dimuat.")
    print("   -> Preprocessor berhasil dimuat.")

    # 3. Muat ulang data bersih dan lakukan train-test split yang sama persis
    # Ini penting untuk memastikan X_test dan y_test konsisten dengan yang digunakan saat training
    df_model_ready = pd.read_csv(data_path)

    # Hapus kolom 'Unnamed: 0' jika masih ada di file CSV sebagai pengaman
    if 'Unnamed: 0' in df_model_ready.columns:
        df_model_ready.drop(columns=['Unnamed: 0'], inplace=True)

    X = df_model_ready.drop(columns=['credit_risk'])
    y = df_model_ready['credit_risk']

    # Gunakan random_state=42 agar splitnya selalu sama
    _, X_test, _, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42, stratify=y
    )
    print("   -> Data uji (X_test, y_test) berhasil dibuat ulang secara konsisten.")

    # 4. Proses X_test menggunakan preprocessor yang sudah dimuat
    X_test_processed = preprocessor.transform(X_test)
    print("   -> X_test berhasil diproses menggunakan preprocessor.")
    print("-" * 60)


    # --- BAGIAN 1: Siapkan DataFrame untuk Analisis Finansial ---
    df_finance = X_test.copy()
    df_finance['credit_risk_actual'] = y_test
    df_finance['credit_risk_predicted'] = champion_model.predict(X_test_processed)

    # --- BAGIAN 2: Hitung Potensi Keuntungan dan Kerugian untuk SETIAP Pinjaman ---
    df_finance['potential_profit'] = (df_finance['int_rate'] / 100) * df_finance['loan_amnt']
    df_finance['potential_loss'] = df_finance['loan_amnt']

    print("✅ DataFrame finansial telah dibuat dengan potensi profit/loss per baris.")
    display(df_finance[['loan_amnt', 'int_rate', 'potential_profit', 'potential_loss', 'credit_risk_actual', 'credit_risk_predicted']].head())

    # --- BAGIAN 3: Hitung Dampak Finansial dari Strategi Model (Tingkat Individu) ---
    print("\n--- Menghitung Dampak Finansial dengan Model (Tingkat Individu) ---")
    profit_from_tn = df_finance[(df_finance['credit_risk_predicted'] == 0) & (df_finance['credit_risk_actual'] == 0)]['potential_profit'].sum()
    loss_from_fn = df_finance[(df_finance['credit_risk_predicted'] == 0) & (df_finance['credit_risk_actual'] == 1)]['potential_loss'].sum()
    net_impact_with_model = profit_from_tn - loss_from_fn

    print(f"   Total Keuntungan dari Pinjaman yang Disetujui & Lunas (TN): ${profit_from_tn:,.2f}")
    print(f"   Total Kerugian dari Pinjaman yang Disetujui & Gagal Bayar (FN): ${loss_from_fn:,.2f}")
    print(f"   => Dampak Finansial Bersih (Dengan Model): ${net_impact_with_model:,.2f}")

    opportunity_cost_fp = df_finance[(df_finance['credit_risk_predicted'] == 1) & (df_finance['credit_risk_actual'] == 0)]['potential_profit'].sum()
    print(f"   (Opportunity Cost dari Penolakan yang Salah (FP): ${opportunity_cost_fp:,.2f})")
    avoided_loss_tp = df_finance[(df_finance['credit_risk_predicted'] == 1) & (df_finance['credit_risk_actual'] == 1)]['potential_loss'].sum()
    print(f"   (Kerugian yang Berhasil Dihindari (TP): ${avoided_loss_tp:,.2f})")

    # --- BAGIAN 4: Hitung Dampak Finansial Skenario Tanpa Model (Setujui Semua) ---
    print("\n--- Menghitung Dampak Finansial Skenario Tanpa Model (Setujui Semua) ---")
    profit_no_model = df_finance[df_finance['credit_risk_actual'] == 0]['potential_profit'].sum()
    loss_no_model = df_finance[df_finance['credit_risk_actual'] == 1]['potential_loss'].sum()
    net_impact_no_model = profit_no_model - loss_no_model

    print(f"   Total Keuntungan jika semua Good Loan disetujui: ${profit_no_model:,.2f}")
    print(f"   Total Kerugian jika semua Bad Loan disetujui: ${loss_no_model:,.2f}")
    print(f"   => Dampak Finansial Bersih (Tanpa Model): ${net_impact_no_model:,.2f}")

    # --- BAGIAN 5: Ringkasan dan Nilai Tambah Model ---
    print("\n" + "="*60)
    print("      RINGKASAN DAMPAK FINANSIAL (ANALISIS TINGKAT INDIVIDU)")
    print("="*60)
    value_add_of_model = net_impact_with_model - net_impact_no_model

    print(f"Dampak Bersih Tanpa Model: ${net_impact_no_model:,.2f}")
    print(f"Dampak Bersih Dengan Model: ${net_impact_with_model:,.2f}")
    print("-" * 60)
    if value_add_of_model > 0:
        print(f"✅ Nilai Tambah Finansial dari Model: ${value_add_of_model:,.2f}")
    else:
        print(f"❌ Model ini secara finansial merugikan sebesar: ${-value_add_of_model:,.2f}")
    print("="*60)

except FileNotFoundError as e:
    print(f"\n❌ GAGAL: File tidak ditemukan. Pastikan path sudah benar dan file ada di Google Drive.")
    print(f"   Detail Error: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 5.3 (Mandiri). Error: {e}")



## SEL 5.4 (DINAMIS): Perbandingan Finansial Otomatis dari Model Tersimpan

In [ ]:
# =============================================================================
# SEL 5.4 (DINAMIS): Perbandingan Finansial Otomatis dari Model Tersimpan
# =============================================================================
print("--- SEL 5.4 (DINAMIS): Memulai Perbandingan Finansial Otomatis ---")

try:
    # --- 1. Definisikan Ulang Asumsi Finansial (Konsisten) ---
    print("💰 Memuat ulang asumsi finansial rata-rata...")
    df_raw_for_finance = pd.read_csv(os.path.join(DATA_RAW_DIR, "loan_data_2007_2014.csv"))

    avg_profit_per_good_loan = df_raw_for_finance[df_raw_for_finance['loan_status'] == 'Fully Paid']['total_rec_int'].mean()
    avg_loss_per_bad_loan = df_raw_for_finance[df_raw_for_finance['loan_status'] == 'Charged Off']['loan_amnt'].mean()

    print(f"   -> Rata-rata Keuntungan per Pinjaman Lunas: ${avg_profit_per_good_loan:,.2f}")
    print(f"   -> Rata-rata Kerugian per Pinjaman Gagal Bayar: ${avg_loss_per_bad_loan:,.2f}")
    print("-" * 60)

    # --- 2. Muat Ulang Data Uji yang Konsisten ---
    # (Pastikan X_test_processed dan y_test sudah ada dari sel sebelumnya, atau muat ulang jika perlu)
    if 'X_test_processed' not in globals() or 'y_test' not in globals():
        print("⚠️  Data uji tidak ditemukan di memori, memuat ulang...")
        # Kode ini adalah pengaman jika Anda menjalankan sel ini di sesi baru
        data_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
        df_model_ready = pd.read_csv(data_path)
        if 'Unnamed: 0' in df_model_ready.columns: df_model_ready.drop(columns=['Unnamed: 0'], inplace=True)
        X = df_model_ready.drop(columns=['credit_risk'])
        y = df_model_ready['credit_risk']
        _, _, _, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

        X_test_processed = np.load(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.npy'))
        print("   -> Data uji berhasil dimuat dari file .npy.")

    # --- 3. Muat SEMUA Model yang Telah Di-tuning & Hitung CM secara Dinamis ---
    print("🤖 Memuat semua model yang telah di-tuning dan menghitung Confusion Matrix...")

    model_names = ['Logistic Regression', 'LightGBM', 'XGBoost', 'CatBoost']
    financial_results = []

    for model_name in model_names:
        safe_model_name = model_name.replace(" ", "_")
        # Sesuaikan nama file jika Anda menggunakan 'champion_model' untuk CatBoost
        if model_name == 'CatBoost' and not os.path.exists(os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')):
             model_path = os.path.join(MODELS_DIR, f'champion_model_{safe_model_name}.joblib')
        else:
             model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')

        try:
            # Muat model
            model = joblib.load(model_path)
            print(f"\n   -> Model '{model_name}' dimuat.")

            # Buat prediksi
            y_pred = model.predict(X_test_processed)

            # Hitung confusion matrix secara otomatis
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
            print(f"      CM: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

            # --- 4. Hitung Dampak Finansial untuk Model Ini ---
            total_profit = tn * avg_profit_per_good_loan
            total_loss = fn * avg_loss_per_bad_loan
            net_impact = total_profit - total_loss
            opportunity_cost = fp * avg_profit_per_good_loan
            avoided_loss = tp * avg_loss_per_bad_loan

            financial_results.append({
                'Model': model_name,
                'Dampak Bersih': net_impact,
                'Total Keuntungan (TN)': total_profit,
                'Total Kerugian (FN)': total_loss,
                'Opportunity Cost (FP)': opportunity_cost,
                'Kerugian Dihindari (TP)': avoided_loss
            })

        except FileNotFoundError:
            print(f"   -> ❌ PERINGATAN: File model untuk '{model_name}' tidak ditemukan di {model_path}. Model ini akan dilewati.")
            continue

    # --- 5. Buat Tabel Perbandingan ---
    if not financial_results:
        print("\n❌ ERROR: Tidak ada model yang berhasil dimuat. Analisis tidak dapat dilanjutkan.")
    else:
        df_comparison = pd.DataFrame(financial_results)
        df_comparison_sorted = df_comparison.sort_values(by='Dampak Bersih', ascending=False)

        # Format angka agar mudah dibaca
        for col in ['Dampak Bersih', 'Total Keuntungan (TN)', 'Total Kerugian (FN)', 'Opportunity Cost (FP)', 'Kerugian Dihindari (TP)']:
            df_comparison_sorted[col] = df_comparison_sorted[col].apply(lambda x: f"${x:,.2f}")

        print("\n" + "="*80)
        print("      TABEL PERBANDINGAN DAMPAK FINANSIAL (DINAMIS)")
        print("="*80)
        display(df_comparison_sorted)
        print("="*80)

        # --- 6. Analisis dan Rekomendasi Otomatis ---
        best_financial_model = df_comparison.sort_values(by='Dampak Bersih', ascending=False).iloc[0]

        print("\n--- Analisis dan Rekomendasi ---")
        print(f"Model terbaik dari perspektif finansial (keuntungan bersih terbesar) adalah '{best_financial_model['Model']}'.")
        print(f"\nModel '{best_financial_model['Model']}' menghasilkan keuntungan bersih sebesar ${best_financial_model['Dampak Bersih']:,.2f}.")

        print("\nREKOMENDASI: Berdasarkan analisis finansial, model '{0}' direkomendasikan untuk implementasi karena memberikan dampak paling positif terhadap keuangan perusahaan.".format(best_financial_model['Model']))

except FileNotFoundError as e:
    print(f"\n❌ ERROR: File data mentah atau data uji tidak ditemukan. Pastikan semua path benar.")
    print(f"   Detail: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 5.4 (Dinamis). Error: {e}")


# Bagian 6: Analisis Robustness

## SEL 6.0 (BARU): Analisis Robustness Model Juara

In [ ]:
# =============================================================================
# SEL 6.0 (BARU): Analisis Robustness Model Juara
# =============================================================================
print("--- SEL 6.0: Memulai Analisis Robustness untuk Model Juara ---")

from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

try:
    # --- 1. Konfigurasi dan Pemuatan Aset ---
    MODEL_TO_VALIDATE = 'CatBoost' # Fokus pada model juara
    N_SPLITS = 10 # Gunakan 10-fold CV untuk analisis yang lebih detail

    print(f"🎯 Menganalisis robustness untuk model: '{MODEL_TO_VALIDATE}'")

    # Muat model yang sudah di-tuning
    safe_model_name = MODEL_TO_VALIDATE.replace(" ", "_")
    model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')
    model = joblib.load(model_path)

    # Muat data yang sudah diproses (sebelum di-split)
    data_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
    df_model_ready = pd.read_csv(data_path)
    if 'Unnamed: 0' in df_model_ready.columns: df_model_ready.drop(columns=['Unnamed: 0'], inplace=True)

    X = df_model_ready.drop(columns=['credit_risk'])
    y = df_model_ready['credit_risk']

    # Muat preprocessor
    preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
    preprocessor = joblib.load(preprocessor_path)

    # Proses seluruh data X
    X_processed = preprocessor.transform(X)
    print("   -> Semua aset (model, data, preprocessor) berhasil dimuat dan diproses.")

    # --- 2. Metode 1: Stabilitas Kinerja pada Cross-Validation ---
    print("\n--- Metode 1: Mengukur Stabilitas Kinerja pada Cross-Validation ---")

    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    fold_metrics = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_processed, y)):
        # Ambil data untuk fold ini
        X_val_fold, y_val_fold = X_processed[val_idx], y.iloc[val_idx]

        # Buat prediksi (model sudah dilatih, kita hanya memvalidasi)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1]

        # Hitung metrik
        auc = roc_auc_score(y_val_fold, y_pred_proba)
        gini = 2 * auc - 1
        ks = calculate_ks(y_val_fold, y_pred_proba)

        fold_metrics.append({'Fold': fold + 1, 'Gini': gini, 'KS': ks})

    df_fold_metrics = pd.DataFrame(fold_metrics)

    print(f"\nKinerja Gini di {N_SPLITS} Folds:")
    display(df_fold_metrics[['Fold', 'Gini']])

    # Hitung statistik stabilitas
    gini_std = df_fold_metrics['Gini'].std()
    print(f"\nStatistik Stabilitas:")
    print(f"   -> Rata-rata Gini: {df_fold_metrics['Gini'].mean():.4f}")
    print(f"   -> Standar Deviasi Gini: {gini_std:.4f}")

    if gini_std < 0.03:
        print("   -> ✅ Stabilitas SANGAT BAIK (Std Dev < 0.03). Model konsisten di berbagai sampel data.")
    elif gini_std < 0.05:
        print("   -> ✅ Stabilitas BAIK (Std Dev < 0.05). Model cukup konsisten.")
    else:
        print("   -> ⚠️  PERINGATAN: Stabilitas KURANG (Std Dev > 0.05). Kinerja model mungkin tidak stabil.")


    # --- 3. Metode 2: Analisis Kinerja pada Segmen Data ---
    print("\n\n--- Metode 2: Mengukur Kinerja pada Segmen Data Penting ---")

    # Pilih beberapa fitur kategorikal penting untuk segmentasi
    # Kita gunakan data X sebelum diproses untuk mendapatkan nilai asli
    segments_to_analyze = ['grade', 'home_ownership', 'purpose']

    # Buat prediksi probabilitas untuk seluruh dataset
    y_full_pred_proba = model.predict_proba(X_processed)[:, 1]

    segment_analysis_results = []

    for segment_col in segments_to_analyze:
        if segment_col in X.columns:
            print(f"\n--- Menganalisis segmen berdasarkan '{segment_col}' ---")
            unique_values = X[segment_col].unique()

            for value in unique_values:
                # Dapatkan indeks untuk segmen spesifik ini
                segment_indices = X[X[segment_col] == value].index

                # Filter y dan prediksi untuk segmen ini
                y_segment = y[segment_indices]
                y_pred_proba_segment = y_full_pred_proba[segment_indices]

                # Pastikan ada cukup data dan kedua kelas ada di segmen
                if len(y_segment) > 100 and len(np.unique(y_segment)) == 2:
                    auc_segment = roc_auc_score(y_segment, y_pred_proba_segment)
                    gini_segment = 2 * auc_segment - 1

                    segment_analysis_results.append({
                        'Segment_Feature': segment_col,
                        'Segment_Value': value,
                        'Gini': gini_segment,
                        'Data_Count': len(y_segment)
                    })

    df_segment_analysis = pd.DataFrame(segment_analysis_results)

    # Tampilkan hasil analisis segmen
    for segment_col in segments_to_analyze:
        if segment_col in df_segment_analysis['Segment_Feature'].values:
            print(f"\nKinerja Gini per Segmen '{segment_col}':")
            display(df_segment_analysis[df_segment_analysis['Segment_Feature'] == segment_col].sort_values(by='Gini', ascending=False))

except FileNotFoundError as e:
    print(f"\n❌ ERROR: File model atau data tidak ditemukan. Pastikan SEL 4.4 sudah dijalankan.")
    print(f"   Detail: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.10. Error: {e}")



## SEL 6.1 (BARU): Analisis Robustness terhadap Noise (Gangguan Data)

In [ ]:
# =============================================================================
# SEL 6.1 (BARU): Analisis Robustness terhadap Noise (Gangguan Data)
# =============================================================================
print("--- SEL 6.1: Memulai Analisis Robustness terhadap Noise ---")

try:
    # --- 1. Konfigurasi dan Pemuatan Aset ---
    MODEL_TO_TEST = 'CatBoost'
    NOISE_LEVELS = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10] # Level noise: 1%, 3%, 5%, 10% dari std dev

    print(f"🎯 Menganalisis ketahanan noise untuk model: '{MODEL_TO_TEST}'")

    # Muat model, preprocessor, dan data uji
    safe_model_name = MODEL_TO_TEST.replace(" ", "_")
    model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')
    model = joblib.load(model_path)

    preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
    preprocessor = joblib.load(preprocessor_path)

    if 'X_test' not in globals() or 'y_test' not in globals():
        print("⚠️  Data uji (X_test) tidak ditemukan, membuat ulang...")
        data_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
        df_model_ready = pd.read_csv(data_path)
        if 'Unnamed: 0' in df_model_ready.columns: df_model_ready.drop(columns=['Unnamed: 0'], inplace=True)
        X = df_model_ready.drop(columns=['credit_risk'])
        y = df_model_ready['credit_risk']
        _, X_test, _, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
        print("   -> Data uji berhasil dibuat ulang.")

    print("   -> Aset (model, data, preprocessor) berhasil dimuat.")

    # --- 2. Hitung Kinerja Baseline (Tanpa Noise) ---
    print("\n--- Menghitung Kinerja Baseline (Tanpa Noise) ---")
    X_test_processed_original = preprocessor.transform(X_test)
    y_pred_proba_original = model.predict_proba(X_test_processed_original)[:, 1]

    baseline_auc = roc_auc_score(y_test, y_pred_proba_original)
    baseline_gini = 2 * baseline_auc - 1

    print(f"   -> Gini Baseline: {baseline_gini:.4f}")

    noise_results = [{'Noise_Level': 0, 'Gini': baseline_gini, 'Gini_Drop': 0.0}]

    # --- 3. Tambahkan Noise dan Evaluasi Ulang ---
    print("\n--- Menambahkan Noise pada Fitur Numerik dan Mengevaluasi Ulang ---")

    numeric_features = X_test.select_dtypes(include=np.number).columns

    for noise_level in NOISE_LEVELS:
        print(f"   -> Menguji dengan tingkat noise: {noise_level*100:.0f}%...")
        X_test_noisy = X_test.copy()

        # Tambahkan noise ke setiap fitur numerik
        for col in numeric_features:
            # Hitung standar deviasi dari kolom
            std_dev = X_test_noisy[col].std()
            # Hasilkan noise acak berdasarkan std dev dan level noise
            noise = np.random.normal(0, std_dev * noise_level, X_test_noisy.shape[0])
            # Tambahkan noise ke data
            X_test_noisy[col] = X_test_noisy[col] + noise

        # Proses data yang sudah terganggu dengan preprocessor yang sama
        X_test_noisy_processed = preprocessor.transform(X_test_noisy)

        # Buat prediksi dengan model
        y_pred_proba_noisy = model.predict_proba(X_test_noisy_processed)[:, 1]

        # Hitung metrik
        noisy_auc = roc_auc_score(y_test, y_pred_proba_noisy)
        noisy_gini = 2 * noisy_auc - 1

        # Hitung penurunan kinerja
        gini_drop = baseline_gini - noisy_gini

        noise_results.append({
            'Noise_Level': noise_level,
            'Gini': noisy_gini,
            'Gini_Drop': gini_drop
        })

    # --- 4. Tampilkan Hasil Analisis Noise ---
    df_noise_analysis = pd.DataFrame(noise_results)
    df_noise_analysis['Gini_Drop_Percent'] = (df_noise_analysis['Gini_Drop'] / baseline_gini) * 100

    print("\n" + "="*80)
    print("      HASIL ANALISIS KETAHANAN TERHADAP NOISE")
    print("="*80)
    display(df_noise_analysis)
    print("="*80)

    # --- 5. Plot Hasil untuk Visualisasi ---
    plt.figure(figsize=(10, 6))
    plt.plot(df_noise_analysis['Noise_Level']*100, df_noise_analysis['Gini'], marker='o', linestyle='-')
    plt.title(f'Kinerja Gini vs. Tingkat Noise untuk Model {MODEL_TO_TEST}')
    plt.xlabel('Tingkat Noise Ditambahkan (%)')
    plt.ylabel('Skor Gini')
    plt.grid(True)
    plt.ylim(bottom=df_noise_analysis['Gini'].min() - 0.02, top=baseline_gini + 0.01) # Atur batas y
    plt.show()

    # --- 6. Berikan Kesimpulan Otomatis ---
    final_drop_percent = df_noise_analysis.iloc[-1]['Gini_Drop_Percent']
    print("\n--- Kesimpulan Robustness terhadap Noise ---")
    if final_drop_percent < 5:
        print(f"   -> ✅ Model SANGAT ROBUST. Penurunan Gini hanya {final_drop_percent:.2f}% bahkan pada tingkat noise 10%.")
    elif final_drop_percent < 10:
        print(f"   -> ✅ Model CUKUP ROBUST. Penurunan Gini sebesar {final_drop_percent:.2f}% pada tingkat noise 10%. Kinerja masih dapat diterima.")
    else:
        print(f"   -> ⚠️  PERINGATAN: Model SENSITIF terhadap noise. Penurunan Gini mencapai {final_drop_percent:.2f}% pada tingkat noise 10%. Perlu hati-hati dalam implementasi.")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: File model atau data tidak ditemukan. Pastikan SEL 4.4 sudah dijalankan.")
    print(f"   Detail: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.11. Error: {e}")



## SEL 6.2 (BARU): Analisis Robustness terhadap Pergeseran Data (PSI)

In [ ]:
# =============================================================================
# SEL 6.2 (BARU): Analisis Robustness terhadap Pergeseran Data (PSI)
# =============================================================================
print("--- SEL 4.12: Memulai Analisis Stabilitas Populasi (PSI) ---")

try:
    # --- 1. Konfigurasi dan Pemuatan Aset ---
    MODEL_TO_VALIDATE = 'CatBoost'
    N_BINS = 10 # Jumlah keranjang untuk kalkulasi PSI

    print(f"🎯 Menganalisis stabilitas populasi untuk model: '{MODEL_TO_VALIDATE}'")

    # Muat model, preprocessor, dan data
    safe_model_name = MODEL_TO_VALIDATE.replace(" ", "_")
    model_path = os.path.join(MODELS_DIR, f'tuned_model_{safe_model_name}.joblib')
    model = joblib.load(model_path)

    preprocessor_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
    preprocessor = joblib.load(preprocessor_path)

    # Kita butuh X_train dan X_test yang asli (sebelum diproses)
    if 'X_train' not in globals() or 'X_test' not in globals():
        print("⚠️  Data latih/uji tidak ditemukan, membuat ulang...")
        data_path = os.path.join(DATA_PROCESSED_DIR, "loan_data_model_ready.csv")
        df_model_ready = pd.read_csv(data_path)
        if 'Unnamed: 0' in df_model_ready.columns: df_model_ready.drop(columns=['Unnamed: 0'], inplace=True)
        X = df_model_ready.drop(columns=['credit_risk'])
        y = df_model_ready['credit_risk']
        X_train, X_test, _, _ = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
        print("   -> Data latih dan uji berhasil dibuat ulang.")

    print("   -> Aset (model, data, preprocessor) berhasil dimuat.")

    # --- 2. Fungsi untuk Menghitung PSI ---
    def calculate_psi(base_series, new_series, bins=N_BINS):
        """Menghitung Population Stability Index (PSI) antara dua series data."""

        # Tentukan batas bin berdasarkan data referensi (base)
        base_series_non_nan = base_series.dropna()
        bin_edges = np.histogram_bin_edges(base_series_non_nan, bins=bins)

        # Hitung persentase di setiap bin untuk data base
        base_counts = np.histogram(base_series_non_nan, bins=bin_edges)[0]
        base_dist = base_counts / len(base_series_non_nan)

        # Hitung persentase di setiap bin untuk data baru
        new_series_non_nan = new_series.dropna()
        new_counts = np.histogram(new_series_non_nan, bins=bin_edges)[0]
        new_dist = new_counts / len(new_series_non_nan)

        # Hindari pembagian dengan nol
        base_dist = np.where(base_dist == 0, 0.0001, base_dist)
        new_dist = np.where(new_dist == 0, 0.0001, new_dist)

        # Hitung PSI
        psi_value = np.sum((new_dist - base_dist) * np.log(new_dist / base_dist))

        return psi_value

    # --- 3. Hitung PSI untuk Skor Model ---
    print("\n--- Menghitung PSI untuk Skor Prediksi Model ---")

    # Dapatkan skor probabilitas untuk data latih dan uji
    X_train_processed = preprocessor.transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    train_scores = pd.Series(model.predict_proba(X_train_processed)[:, 1])
    test_scores = pd.Series(model.predict_proba(X_test_processed)[:, 1])

    model_psi = calculate_psi(train_scores, test_scores)

    print(f"   -> PSI Skor Model: {model_psi:.4f}")
    if model_psi < 0.1:
        print("   -> ✅ Stabilitas SANGAT BAIK. Distribusi skor model konsisten antara data latih dan uji.")
    elif model_psi < 0.25:
        print("   -> ✅ Stabilitas CUKUP. Ada sedikit pergeseran pada distribusi skor.")
    else:
        print("   -> ⚠️  PERINGATAN: Stabilitas BURUK. Distribusi skor telah bergeser signifikan.")

    # --- 4. Hitung CSI untuk Fitur-fitur Penting ---
    print("\n--- Menghitung CSI (Characteristic Stability Index) untuk Fitur Penting ---")

    features_to_check = ['int_rate', 'loan_amnt', 'dti', 'annual_inc', 'credit_history_length_years']
    csi_results = []

    for feature in features_to_check:
        if feature in X_train.columns:
            csi_value = calculate_psi(X_train[feature], X_test[feature])
            csi_results.append({'Feature': feature, 'CSI': csi_value})

    df_csi = pd.DataFrame(csi_results)

    print("\nHasil Analisis CSI:")
    display(df_csi.sort_values(by='CSI', ascending=False))

    # Berikan kesimpulan umum
    if (df_csi['CSI'] > 0.25).any():
        print("\n⚠️  PERINGATAN: Ditemukan fitur dengan pergeseran distribusi yang signifikan (CSI > 0.25).")
        print("   Fitur-fitur ini perlu diinvestigasi lebih lanjut karena perilakunya berubah antara data latih dan uji.")
    else:
        print("\n✅ Stabilitas fitur-fitur utama terlihat baik (semua CSI < 0.25).")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: File model atau data tidak ditemukan. Pastikan SEL 4.4 sudah dijalankan.")
    print(f"   Detail: {e}")
except Exception as e:
    print(f"\n❌ GAGAL pada SEL 4.12. Error: {e}")

